In [1]:
!pip install numpy scipy matplotlib pandas pillow scikit-learn gempy gempy-viewer autograd pyvista vtk tqdm --quiet

In [13]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk,font
from PIL import Image, ImageTk
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.cluster import KMeans
import json
import gempy as gp
import gempy_viewer as gpv
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from autograd import numpy as np
import numpy as npy
import pyvista as pv
from autograd import elementwise_grad as egrad
import vtk
import warnings
import os
import threading
import sys
import itertools
from tqdm import tqdm
import traceback
import time
from scipy.optimize import minimize
from scipy.constants import G
from scipy.ndimage import gaussian_filter
from scipy.interpolate import Rbf
from sympy import symbols, diff
from scipy.interpolate import RegularGridInterpolator
import cmcrameri as cmc

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)


class GeologicalModelApp:
    def __init__(self, root):
        self.root = root
        self.root.geometry("1920x1080")
        self.menu_font = font.Font(family='Arial', size=16)
        self.root.option_add('*Menu.font', self.menu_font)
        self.menubar = tk.Menu(root,)
        root.config(menu=self.menubar)
        self.function_menu = tk.Menu(self.menubar, tearoff=0)
        self.menubar.add_cascade(label="Model generator", menu=self.function_menu)
        self.process_menu = tk.Menu(self.menubar, tearoff=0)
        self.menubar.add_cascade(label="Model Process", menu=self.process_menu)
        self.interpolation_menu = tk.Menu(self.menubar, tearoff=0)
        self.menubar.add_cascade(label="Interpolation and Comparison", menu=self.interpolation_menu)
        self.menubar.entryconfig("Model generator", font=self.menu_font)
        self.function_menu.add_command(label="save/load model", command=lambda: self.show_tab(self.tab1, 'Save/Load Data'))
        self.function_menu.add_command(label="Create GemPy Model", command=lambda: self.show_tab(self.tab2, 'Create GemPy Model'))
        self.function_menu.add_command(label="Drawing", command=lambda: self.show_tab(self.tab3, 'Drawing'))
        self.function_menu.add_command(label="2D and 3D Plot", command=lambda: self.show_tab(self.tab4, '2D and 3D Plot'))
        self.process_menu.add_command(label="VTK to Scalar Field", command=lambda: self.show_tab(self.tab5, 'VTK to Scalar Field'))
        self.interpolation_menu.add_command(label="Implicit Interpolation", command=lambda: self.show_tab(self.tab6, 'Implicit Interpolation'))
        self.process_menu.add_command(label="Cross-Validation", command=lambda: self.show_tab(self.tab7, 'Cross-Validation'))
        self.interpolation_menu.add_command(label="Scalar Field Comparison", command=lambda: self.show_tab(self.tab8, 'Scalar Field Comparison'))
        self.process_menu.add_command(label="Gravity Simulation", command=lambda: self.show_tab(self.tab11, 'Gravity Simulation'))        
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill='both', expand=True)

        self.style = ttk.Style()
        self.style.configure('TNotebook.Tab', font=('Arial', '16'))
        self.style.configure('TButton', font=('Arial', 14), padding=10, borderwidth=10, relief="flat", background="#C0E6A7")
 
        
        self.tab1 = ttk.Frame(self.notebook)
        self.tab2 = ttk.Frame(self.notebook)
        self.tab3 = ttk.Frame(self.notebook)
        self.tab4 = ttk.Frame(self.notebook)
        self.tab5 = ttk.Frame(self.notebook)
        self.tab6 = ttk.Frame(self.notebook)
       
        self.auto_corr = tk.IntVar()
        self.auto_corr.set(0)
        self.show_drift_var = tk.IntVar()
        self.show_drift_var.set(0)
        self.only_show_data = tk.IntVar()
        self.only_show_data.set(0)
        self.name = {}
        self.name_saved = []
        self.x_pos = {}
        self.x_pos_saved = []
        self.y_pos = {}
        self.y_pos_saved = []
        self.x_points = []
        self.y_points = []
        self.vertices_df = None
        self.normals_df = None

        # Define font styles
        self.title_font = ("Helvetica", 16, "bold")
        self.label_font = ("Helvetica", 14)
        self.label_font_s = ("Helvetica", 10)
        self.entry_font = ("Helvetica", 14)
        self.button_font = ("Helvetica", 14, "bold")

        # Tab 1: Save/Load Data
        self.save_button = ttk.Button(self.tab1, text="Save Data",  command=self.save_data, style='TButton')
        self.save_button.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_button = ttk.Button(self.tab1, text="Load Data",  command=self.load_data, style='TButton')
        self.load_button.grid(row=0, column=1, padx=10, pady=10, sticky='w')
        
        self.save_model_button = ttk.Button(self.tab1, text="Save Gempy Model",  command=self.save_gempy_model, style='TButton')
        self.save_model_button.grid(row=0, column=2, padx=10, pady=10, sticky='w')

        # Tab 2: Model Parameters
        self.Layer_number_label = tk.Label(self.tab2, text="Please enter the layer number:", font=self.label_font)
        self.layer_number_entry = tk.Entry(self.tab2, font=self.entry_font)
        self.layer_number_confirm = ttk.Button(self.tab2, text="Confirm", style='TButton', command=self.set_layer_name)

        self.Layer_number_label.grid(row=0, column=0, padx=10, pady=10, sticky='w')
        self.layer_number_entry.grid(row=0, column=1, padx=10, pady=10, sticky='w')
        self.layer_number_confirm.grid(row=0, column=2, padx=10, pady=10, sticky='w')

        self.en = tk.Checkbutton(self.tab2, text="Enable auto correction", variable=self.auto_corr, onvalue=1, offvalue=0, font=self.label_font)
        self.en.grid(row=1, column=0, padx=10, pady=10, sticky='w')

        # Tab 3: Drawing
        self.draw_frame = tk.Frame(self.tab3)
        self.draw_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        # Tab 4: 2D Plot
        self.plot_2d_frame = tk.Frame(self.tab4)
        self.plot_2d_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.plot_3d_button = ttk.Button(self.plot_2d_frame, text="Show 3D Plot", style='TButton', command=self.plot_3d)
        self.plot_3d_button.pack(pady=10)

        # Tab 5: VTK Operations
        self.vtk_operations_frame = tk.Frame(self.tab5)
        self.vtk_operations_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_vtk_button = ttk.Button(self.vtk_operations_frame, text="Load VTK File", style='TButton', command=self.load_vtk_file)
        self.load_vtk_button.grid(row=0, column=0, padx=10, pady=10, sticky='w')
        
        self.compute_scalar_button = ttk.Button(self.vtk_operations_frame, text="Compute Scalar Field", style='TButton', command=self.compute_scalar_field)
        self.compute_scalar_button.grid(row=0, column=1, padx=10, pady=10, sticky='w')
        
        self.save_scalar_button = ttk.Button(self.vtk_operations_frame, text="Save Scalar Field", style='TButton', command=self.save_scalar_field)
        self.save_scalar_button.place(relx=0.68, rely=0.19, anchor='center')

        self.vertices_file_path = tk.StringVar()
        self.normals_file_path = tk.StringVar()
        self.scalar_file_path = tk.StringVar()

        tk.Label(self.vtk_operations_frame, text="Vertices File Path:", font=self.label_font).grid(row=1, column=0, padx=10, pady=5, sticky='w')
        self.vertices_entry = tk.Entry(self.vtk_operations_frame, textvariable=self.vertices_file_path, font=self.entry_font, width=50)
        self.vertices_entry.grid(row=1, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.vtk_operations_frame, text="Normals File Path:", font=self.label_font).grid(row=2, column=0, padx=10, pady=5, sticky='w')
        self.normals_entry = tk.Entry(self.vtk_operations_frame, textvariable=self.normals_file_path, font=self.entry_font, width=50)
        self.normals_entry.grid(row=2, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.vtk_operations_frame, text="Scalar Field File Path:", font=self.label_font).grid(row=3, column=0, padx=10, pady=5, sticky='w')
        self.scalar_entry = tk.Entry(self.vtk_operations_frame, textvariable=self.scalar_file_path, font=self.entry_font, width=50)
        self.scalar_entry.grid(row=3, column=1, padx=10, pady=5, sticky='w')

        self.save_excel_button = ttk.Button(self.vtk_operations_frame, text="Save to Excel", style='TButton', command=self.save_vtk_to_excel)
        self.save_excel_button.grid(row=4, column=0, padx=10, pady=10, sticky='w')
        self.save_excel_button.place(relx=0.90, rely=0.19, anchor='center')

        self.pv_canvas_frame = tk.Frame(self.tab5, width=800, height=600)
        self.pv_canvas_frame.grid(row=5, column=0, columnspan=3, padx=10, pady=10)

        self.plotter = None
        self.init_pyvista_plotter()

        # Tab 6: RBF Interpolation
        self.rbf_frame = tk.Frame(self.tab6)
        self.rbf_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_surface_button = ttk.Button(self.rbf_frame, text="Load Surface Points", style='TButton', command=self.load_surface_points)
        self.load_surface_button.grid(row=0, column=0, padx=10, pady=10, sticky='w')
        
        self.load_orientation_button = ttk.Button(self.rbf_frame, text="Load Orientation Points", style='TButton', command=self.load_orientation_points)
        self.load_orientation_button.grid(row=0, column=1, padx=10, pady=10, sticky='w')

        self.load_orientations_button = ttk.Button(self.rbf_frame, text="Load Orientations", style='TButton', command=self.load_orientations)
        self.load_orientations_button.grid(row=0, column=2, padx=10, pady=10, sticky='w')

        tk.Label(self.rbf_frame, text="Sampling method:", font=self.label_font).grid(row=1, column=2, padx=10, pady=5, sticky='w')
        self.Sampling_method = ttk.Combobox(self.rbf_frame, font=self.entry_font, values=[
            'Random','Systematic' ,'Statistic(K-means)',])
        self.Sampling_method.grid(row=1, column=3, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Number of Sample Surface Points:", font=self.label_font).grid(row=1, column=0, padx=10, pady=5, sticky='w')
        self.num_surface_points = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.num_surface_points.grid(row=1, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Number of Sample Orientation Points:", font=self.label_font).grid(row=2, column=0, padx=10, pady=5, sticky='w')
        self.num_orientation_points = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.num_orientation_points.grid(row=2, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Kernel Function:", font=self.label_font).grid(row=3, column=0, padx=10, pady=5, sticky='w')
        self.kernel_function = ttk.Combobox(self.rbf_frame, font=self.entry_font, values=[
            'cubic', 'gaussian', 'quintic', 'cubic_covariance', 'multiquadric', 'linear', 
            'thin_plate', 'inverse_quadratic', 'inverse_multiquadric', 'gaussian_covariance', 
            'ga_cub_cov', 'ga_cub'])
        self.kernel_function.grid(row=3, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Drift Type:", font=self.label_font).grid(row=4, column=0, padx=10, pady=5, sticky='w')
        self.drift_type = ttk.Combobox(self.rbf_frame, font=self.entry_font, values=[
            'None', 'First Order Polynomial','Second Order Polynomial', 'Ellipsoid', 'Dome Shaped','2D Custom','3D Custom'])
        self.drift_type.grid(row=4, column=1, padx=10, pady=5, sticky='w')

        self.load_drift_scalar_button = ttk.Button(self.rbf_frame, text="Load Drift Scalar Field", style='TButton', command=self.load_drift_scalar)
        self.load_drift_scalar_button.grid(row=4, column=2, padx=10, pady=10, sticky='w')

        tk.Label(self.rbf_frame, text="Drift scaling factor:", font=self.label_font).grid(row=4, column=3, padx=10, pady=5, sticky='w')
        self.scaling_factor = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.scaling_factor.grid(row=4, column=4, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Epsilon for gaussian kernel (default is calculated):", font=self.label_font).grid(row=5, column=0, padx=10, pady=5, sticky='w')
        self.epsilon_entry = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.epsilon_entry.grid(row=5, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Range for covariance kernel (default is calculated):", font=self.label_font).grid(row=6, column=0, padx=10, pady=5, sticky='w')
        self.range_entry = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.range_entry.grid(row=6, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Parameter a:the width of the salt dome or x coordinate of sphere center  (default:1)", font=self.label_font).grid(row=7, column=0, padx=10, pady=5, sticky='w')
        self.param_a_entry = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.param_a_entry.grid(row=7, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Parameter b:flatness of the top of the salt dome or y coordinate of sphere center (default:1):", font=self.label_font).grid(row=8, column=0, padx=10, pady=5, sticky='w')
        self.param_b_entry = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.param_b_entry.grid(row=8, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.rbf_frame, text="Parameter c:height of the salt dome or z coordinate of sphere center (default:1):", font=self.label_font).grid(row=9, column=0, padx=10, pady=5, sticky='w')
        self.param_c_entry = tk.Entry(self.rbf_frame, font=self.entry_font)
        self.param_c_entry.grid(row=9, column=1, padx=10, pady=5, sticky='w')

        self.compute_rbf_button = ttk.Button(self.rbf_frame, text="Compute RBF Interpolation", style='TButton', command=self.compute_rbf_interpolation)
        self.compute_rbf_button.grid(row=10, column=0, padx=10, pady=10, sticky='w')

        self.show_drift = tk.Checkbutton(self.rbf_frame, text="show drift contribution", variable=self.show_drift_var, onvalue=1, offvalue=0, font=self.label_font)
        self.show_drift.grid(row=11, column=0, padx=10, pady=10, sticky='w')

        self.show_data = tk.Checkbutton(self.rbf_frame, text="only show data", variable=self.only_show_data, onvalue=1, offvalue=0, font=self.label_font)
        self.show_data.grid(row=12, column=0, padx=10, pady=10, sticky='w')

        tk.Label(self.rbf_frame, text="Model color:", font=self.label_font).grid(row=13, column=0, padx=10, pady=5, sticky='w')
        self.model_color = ttk.Combobox(self.rbf_frame, font=self.entry_font, values=[
            'red', 'green', 'blue', 'orange', 'purple', 'black', 'darkgrey'])
        self.model_color.grid(row=13, column=1, padx=10, pady=5, sticky='w')


        # Tab 7: Cross-Validation
        self.tab7 = ttk.Frame(self.notebook)
        self.create_cross_validation_tab()

        # Tab 8：Scalar Field Comparison
        self.scalar1 = None
        self.scalar2 = None
        self.use_flip = tk.BooleanVar()
        self.use_reverse = tk.BooleanVar()
        self.tab8 = ttk.Frame(self.notebook)
        self.create_scalar_comparison_tab()

        # Tab 9 : About
        self.tab9 = ttk.Frame(self.notebook)

        # Tab 11: Gravity simulation
        self.tab11 = ttk.Frame(self.notebook)
        self.gravity_simulation_frame = tk.Frame(self.tab11)
        self.gravity_simulation_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')
        self.load_vtk_button = ttk.Button(self.gravity_simulation_frame, text="Load VTK File", style='TButton', command=self.load_vtk_to_gravity)
        self.load_vtk_button.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.gravity_fwd_button = ttk.Button(self.gravity_simulation_frame, text="Compute Gravity Forward Scalar Field", style='TButton', command=self.gravity_fwd_2d)
        self.gravity_fwd_button.grid(row=0, column=1, padx=10, pady=10, sticky='w')

        self.gravity_canvas_frame = tk.Frame(self.tab11)
        self.gravity_canvas_frame.grid(row=4, column=0, padx=10, pady=10, columnspan=4)

        # Load the image
        image_path = "workflow.png"  # Replace with the actual path to your image
        image = Image.open(image_path)
        photo = ImageTk.PhotoImage(image)
        text_label = tk.Label(self.tab9, text="This software is designed for kernel based implicit geological modeling.\n"
                         "The functions include:\n"
                         "1. Create Gempy model by drawing.\n"
                         "2. Save and load existing drawing model, or save GemPy model as .vtp format "
                         "(only for single layer model).\n"
                         "3. Visualize created GemPy model in 2D and 3D.\n"
                         "4. Create scalar field for .vtp or .vtk model, and save the result as .npy format. "
                         "The vertices and normals of model also can be saved to excel file.\n"
                         "5. Implicit modeling using surface points and orientations. The sampling density can be changed.\n"
                         "6. Cross-validation method for finding the best parameter of kernels and drift.\n"
                         "7. Scalar field comparison using cosine difference.\n"
                         "The meaning of parameter a, b, c in drift function:\n"
                         "For dome shaped drift:\n"
                         "Parameter 𝑎: This parameter controls the width of the salt dome. A larger value of 𝑎 results in a wider lateral extension of the salt dome, while a smaller value of 𝑎 leads to a narrower width.\n"
                         "Parameter 𝑏: This parameter governs the flatness of the salt dome’s top. A larger value of 𝑏 makes the top more flat and extended, whereas a smaller value of 𝑏 leads to a steeper top.\n"
                         "Parameter 𝑐: This parameter determines the height of the salt dome. A larger value of 𝑐 results in a taller salt dome, while a smaller value of 𝑐 produces a lower structure.\n"
                         "For ellipsoid drift:\n"
                         "Parameter 𝑎: Semi axis along x-direction.\n"
                         "Parameter 𝑏: Semi axis along y-direction.\n"
                         "Parameter 𝑐: Semi axis along z-direction.\n",
         font=self.label_font,anchor='w', justify='left',wraplength=1000)
        # Create the image label
        image_label = tk.Label(self.tab9, image=photo)

        # Place the labels in the grid
        text_label.grid(row=0, column=0, padx=10, pady=10, sticky='w')
        image_label.grid(row=0, column=1, padx=10, pady=10, sticky='e')

        # Keep a reference to the image to prevent it from being garbage collected
        image_label.image = photo

        # Tab 10 : workflow
        self.tab10 = ttk.Frame(self.notebook)
        self.notebook.add(self.tab10, text='Workflow')
        self.notebook.add(self.tab9, text='Help')
        # Load the image
        image_path = "workguide_2.png"  # Replace with the actual path to your image
        image = Image.open(image_path)
        original_width, original_height = image.size
        scale_factor = min(1850 / original_width, 1080 / original_height)
        new_width = int(original_width * scale_factor)
        new_height = int(original_height * scale_factor)
        image = image.resize((new_width, new_height), Image.LANCZOS)
        photo = ImageTk.PhotoImage(image)
        
        # Create the image label
        image_label = tk.Label(self.tab10, image=photo)

        # Place the labels in the grid
        image_label.place(x=0, y=0, relwidth=1, relheight=1) 

        # Keep a reference to the image to prevent it from being garbage collected
        image_label.image = photo
        tab1_p = ImageTk.PhotoImage(Image.open("tab1.png").resize((300, 55), Image.LANCZOS))
        tab2_p = ImageTk.PhotoImage(Image.open("tab2.png").resize((300, 55), Image.LANCZOS))
        tab3_p = ImageTk.PhotoImage(Image.open("tab3.png").resize((300, 55), Image.LANCZOS))
        tab5_p = ImageTk.PhotoImage(Image.open("tab5.png").resize((300, 77), Image.LANCZOS))
        tab5_2_p = ImageTk.PhotoImage(Image.open("tab5_2.png").resize((300, 55), Image.LANCZOS))
        tab6_p = ImageTk.PhotoImage(Image.open("tab6.png").resize((300, 55), Image.LANCZOS))
        tab7_p = ImageTk.PhotoImage(Image.open("tab7.png").resize((320, 55), Image.LANCZOS))
        tab8_1_p = ImageTk.PhotoImage(Image.open("tab8_1.png").resize((300, 55), Image.LANCZOS))
        tab8_2_p = ImageTk.PhotoImage(Image.open("tab8_2.png").resize((300, 55), Image.LANCZOS))
        tab8_3_p = ImageTk.PhotoImage(Image.open("tab8_3.png").resize((300, 55), Image.LANCZOS))

        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.12, rely=0.47, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab2_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab2, 'Create GemPy Model'))
        canvas.image = tab2_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.12, rely=0.55, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab3_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab3, 'Drawing'))
        canvas.image = tab3_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.12, rely=0.63, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab1_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab1, 'Save/Load Data'))
        canvas.image = tab1_p
        canvas = tk.Canvas(self.tab10, width=300, height=70, highlightthickness=0)
        canvas.place(relx=0.395, rely=0.31, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab5_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab5, 'VTK to Scalar Field'))
        canvas.image = tab5_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.395, rely=0.39, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab5_2_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab5, 'VTK to Scalar Field'))
        canvas.image = tab5_2_p
        canvas = tk.Canvas(self.tab10, width=320, height=55, highlightthickness=0)
        canvas.place(relx=0.65, rely=0.195, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab7_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab7, 'Cross-Validation'))
        canvas.image = tab7_p
        canvas = tk.Canvas(self.tab10, width=320, height=55, highlightthickness=0)
        canvas.place(relx=0.89, rely=0.195, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab7_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab6, 'Implicit Interpolation'))
        canvas.image = tab7_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.885, rely=0.86, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab5_2_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab5, 'VTK to Scalar Field'))
        canvas.image = tab5_2_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.395, rely=0.67, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab8_1_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab8, 'Scalar Field Comparison'))
        canvas.image = tab8_1_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.395, rely=0.75, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab8_2_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab8, 'Scalar Field Comparison'))
        canvas.image = tab8_2_p
        canvas = tk.Canvas(self.tab10, width=300, height=55, highlightthickness=0)
        canvas.place(relx=0.395, rely=0.83, anchor='center') 
        canvas_image = canvas.create_image(0, 0, anchor='nw', image=tab8_3_p)
        canvas.tag_bind(canvas_image, "<Button-1>", lambda event: self.show_tab(self.tab8, 'Scalar Field Comparison'))
        canvas.image = tab8_3_p


    def show_tab(self, tab, tab_text):
        for t, text in [(self.tab1, 'Save/Load Data'), (self.tab2, 'Create GemPy Model'),
                        (self.tab3, 'Drawing'), (self.tab4, '2D and 3D Plot'), (self.tab5, 'VTK to Scalar Field'),
                        (self.tab6, 'Implicit Interpolation'), (self.tab7, 'Cross-Validation'), (self.tab8, 'Scalar Field Comparison'),
                        (self.tab11, 'Gravity Simulation')]:
            if text in [self.notebook.tab(nt, "text") for nt in self.notebook.tabs()]:
                self.notebook.forget(t)
        self.notebook.insert(0, tab, text=tab_text)
        self.notebook.select(tab)



    def init_pyvista_plotter(self):
        pass  

    def combine_funcs(self, *funcs):
        def inner_combined_func(*args, **kwargs):
            for f in funcs:
                f(*args, **kwargs)
        return inner_combined_func

    def set_layer_name(self):
        for i in range(int(self.layer_number_entry.get())):
            tk.Label(self.tab2, text="Layer " + str(i + 1) + " name:", font=self.label_font).grid(row=i + 2, column=0, padx=10, pady=5, sticky='w')
            layer_name = f'Layer {str(i + 1)} name'
            self.name[layer_name] = tk.Entry(self.tab2, font=self.entry_font)
            self.name[layer_name].grid(row=i + 2, column=1, padx=10, pady=5, sticky='w')

            tk.Label(self.tab2, text="Layer " + str(i + 1) + " X section Position:", font=self.label_font).grid(row=i + 2, column=2, padx=10, pady=5, sticky='w')
            self.x_pos[layer_name] = tk.Entry(self.tab2, font=self.entry_font)
            self.x_pos[layer_name].grid(row=i + 2, column=3, padx=10, pady=5, sticky='w')

            tk.Label(self.tab2, text="Layer " + str(i + 1) + " Y section Position:", font=self.label_font).grid(row=i + 2, column=4, padx=10, pady=5, sticky='w')
            self.y_pos[layer_name] = tk.Entry(self.tab2, font=self.entry_font)
            self.y_pos[layer_name].grid(row=i + 2, column=5, padx=10, pady=5, sticky='w')

        tk.Button(self.tab2, text="Draw!", font=self.button_font, command=self.combine_funcs(self.save_name, self.set_draw, lambda: self.show_tab(self.tab3, 'Drawing'))).grid(row=i + 3, column=0, padx=10, pady=10, sticky='w')

    def save_name(self):
        for i in range(int(self.layer_number_entry.get())):
            self.name_saved.append(self.name[f'Layer {str(i + 1)} name'].get())

        for i in range(int(self.layer_number_entry.get())):
            self.x_pos_saved.append(self.x_pos[f'Layer {str(i + 1)} name'].get())

        for i in range(int(self.layer_number_entry.get())):
            self.y_pos_saved.append(self.y_pos[f'Layer {str(i + 1)} name'].get())

    def draw_X(self):
        %run draw.py

        with open("curve_points.json", "r") as file:
            points = json.load(file)

        slice_x = np.array(points)[-1]
        self.x_points.append(slice_x)

        fig = plt.Figure(figsize=(5, 4), dpi=100)
        ax = fig.add_subplot()

        for i in range(len(self.x_points)):
            ax.plot(self.x_points[i][:, 0], self.x_points[i][:, 1], "-")
        ax.set_xlim(0, 800)
        ax.set_ylim(0, 600)
        ax.grid()
        ax.invert_yaxis()

        canvas = FigureCanvasTkAgg(fig, master=self.draw_frame)
        canvas.draw()
        canvas.get_tk_widget().grid(row=int(self.layer_number_entry.get()) + 10, column=0, rowspan=1, columnspan=5, padx=10, pady=10)

    def draw_Y(self):
        %run draw.py

        with open("curve_points.json", "r") as file:
            points = json.load(file)
        slice_y = np.array(points)[-1]
        self.y_points.append(slice_y)

        fig = plt.Figure(figsize=(5, 4), dpi=100)
        ax = fig.add_subplot()

        for i in range(len(self.y_points)):
            ax.plot(self.y_points[i][:, 0], self.y_points[i][:, 1], "-")
        ax.set_xlim(0, 800)
        ax.set_ylim(0, 600)
        ax.grid()
        ax.invert_yaxis()

        canvas = FigureCanvasTkAgg(fig, master=self.draw_frame)
        canvas.draw()
        canvas.get_tk_widget().grid(row=int(self.layer_number_entry.get()) + 10, column=5, rowspan=1, columnspan=5, padx=10, pady=10)

    def set_draw(self):
        for i in range(int(self.layer_number_entry.get())):
            ttk.Button(self.draw_frame, text="Please draw Layer " + str(i + 1) + " X cross-section", style='TButton', command=self.draw_X).grid(row=i, column=0, padx=10, pady=5, sticky='w')
            ttk.Button(self.draw_frame, text="Please draw Layer " + str(i + 1) + " Y cross-section", style='TButton', command=self.draw_Y).grid(row=i, column=1, padx=10, pady=5, sticky='w')

        ttk.Button(self.tab3, text="Create model!", style='TButton', command=self.create_model).place(relx=0.6, rely=0.015, )

    def surface_all(self, auto_corr):
        surface_all = pd.DataFrame(columns=('X', 'Y', 'Z', 'formation'))
        for i in range(np.array(self.x_points).shape[0]):
            slice_x = np.array(self.x_points)[i]
            slice_y = np.array(self.y_points)[i]
            slice_x_max = np.array([slice_x[:, 0][slice_x[:, 1] == slice_x[:, 1].min()][0], slice_x[:, 1].min()])
            slice_y_max = np.array([slice_y[:, 0][slice_y[:, 1] == slice_y[:, 1].min()][0], slice_y[:, 1].min()])

            if auto_corr:
                diff = slice_x_max - slice_y_max
                slice_y[:, 1] += diff[1]

            x_slice = [slice_x[:, 0],
                       np.ones(len(slice_x)) * int(self.x_pos_saved[i]),
                       -slice_x[:, 1]]
            x_slice = pd.DataFrame((np.array(x_slice).T), columns=('X', 'Y', 'Z'))

            y_slice = [np.ones(len(slice_y)) * int(self.y_pos_saved[i]),
                       slice_y[:, 0],
                       -slice_y[:, 1]]
            y_slice = pd.DataFrame((np.array(y_slice).T), columns=('X', 'Y', 'Z'))

            formation = pd.DataFrame(data={'formation': [self.name_saved[i]] * (len(x_slice) + len(y_slice))})

            surface = pd.concat([pd.concat([x_slice, y_slice], ignore_index=True), formation], axis=1)

            surface_all = pd.concat([surface_all, surface], ignore_index=True)
        return surface_all

    def cal_orientation(self, x, y):
        x1 = y[1] - x[1]
        y1 = y[0] - x[0]
        return x1 / np.sqrt(x1 ** 2 + y1 ** 2), y1 / np.sqrt(x1 ** 2 + y1 ** 2)

    def save_data(self):
        data = {
            "name_saved": self.name_saved,
            "x_pos_saved": self.x_pos_saved,
            "y_pos_saved": self.y_pos_saved,
            "x_points": [points.tolist() for points in self.x_points],
            "y_points": [points.tolist() for points in self.y_points]
        }
        file_path = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if file_path:
            with open(file_path, "w") as file:
                json.dump(data, file)

    def load_data(self):
        file_path = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
        if file_path:
            with open(file_path, "r") as file:
                data = json.load(file)
            self.name_saved = data["name_saved"]
            self.x_pos_saved = data["x_pos_saved"]
            self.y_pos_saved = data["y_pos_saved"]
            self.x_points = [np.array(points) for points in data["x_points"]]
            self.y_points = [np.array(points) for points in data["y_points"]]
            messagebox.showinfo("Load Data", "Data loaded successfully!")
            self.create_model()  # Automatically create model after loading data

    def save_gempy_model(self):
        directory = filedialog.askdirectory()
        if directory:
            geo_model = self._create_model_logic(self.auto_corr.get())
            sol = gp.compute_model(gempy_model=geo_model,engine_config=gp.data.GemPyEngineConfig(use_gpu=False,
                                                            dtype='float32', backend=gp.data.AvailableBackends.PYTORCH))

            # gpv = gp.plot_3d(geo_model, image=False, plotter_type='basic', show_data=0)
            
            surface_points_path = os.path.join(directory, 'surface_points.csv')
            orientations_path = os.path.join(directory, 'orientations.csv')
            
            geo_model.surface_points_copy.df.to_csv(surface_points_path)
            geo_model.orientations_copy.df.to_csv(orientations_path)
            
            for name in self.name_saved:
                mesh_path = os.path.join(directory, f'{name}.vtp')
                try:
                    gpv.plot_3d(geo_model, image=False, plotter_type='basic', show_data=0).surface_poly[name].save(mesh_path)
                except KeyError as e:
                    print(f"Error: {e}")
                    messagebox.showerror("Save Error", f"Could not save {name}.vtp. Please check the surface name.")
                    continue
            
            messagebox.showinfo("Save Gempy Model", "Gempy model saved successfully!")

    def load_vtk_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("VTK files", "*.vtp"), ("VTK files", "*.vtk")])
        if file_path:
            self.mesh = pv.read(file_path)
            self.vertices_file_path.set(file_path.replace(".vtp", "_points.xlsx"))
            self.normals_file_path.set(file_path.replace(".vtp", "_normals.xlsx"))
            self.scalar_file_path.set(file_path.replace(".vtp", "_scalar.npy"))
            self.vertices_file_path.set(file_path.replace(".vtk", "_points.xlsx"))
            self.normals_file_path.set(file_path.replace(".vtk", "_normals.xlsx"))
            self.scalar_file_path.set(file_path.replace(".vtk", "_scalar.npy"))
            messagebox.showinfo("VTK Operations", "VTK file loaded successfully!")

    def generate_normals(self, polydata):
        normal_generator = vtk.vtkPolyDataNormals()
        normal_generator.SetInputData(polydata)
        normal_generator.ComputePointNormalsOn()
        normal_generator.ComputeCellNormalsOff()
        normal_generator.Update()
        return normal_generator.GetOutput()

    def get_vertices_and_normals(self, mesh):
        surface_mesh = mesh.extract_surface()
        polydata = surface_mesh

        # Generate normals if not present
        polydata_with_normals = self.generate_normals(polydata)

        # Get points (vertices)
        points = polydata_with_normals.GetPoints()
        vertices = []
        for i in range(points.GetNumberOfPoints()):
            vertices.append(points.GetPoint(i))

        # Get normals
        normals_array = polydata_with_normals.GetPointData().GetNormals()
        normals = []
        for i in range(normals_array.GetNumberOfTuples()):
            normals.append(normals_array.GetTuple(i))

        return vertices, normals

    def save_to_excel(self, vertices, normals, vertices_file, normals_file):
        # Create DataFrames
        vertices_df = pd.DataFrame(vertices, columns=['X', 'Y', 'Z'])
        normals_df = pd.DataFrame(normals, columns=['x', 'y', 'z'])

        # Save to Excel files
        vertices_df.to_excel(vertices_file, index=False)
        normals_df.to_excel(normals_file, index=False)

    def save_vtk_to_excel(self):
        vertices, normals = self.get_vertices_and_normals(self.mesh)
        self.save_to_excel(vertices, normals, self.vertices_file_path.get(), self.normals_file_path.get())
        messagebox.showinfo("VTK Operations", "Vertices and normals saved to Excel successfully!")

    def compute_scalar_field(self):
        scalar_field = self.vtk_to_scalar(self.mesh)
        self.scalar_field = scalar_field  # Store the scalar field for saving later
        self.plot_scalar_field_in_gui(scalar_field)

    def vtk_to_scalar(self, mesh):
        surface_mesh = mesh.extract_surface()

        # Create a uniform grid surrounding the mesh
        x_min, x_max, y_min, y_max, z_min, z_max = surface_mesh.bounds
        padding = 10  # add some padding to the grid dimensions
        grid = pv.ImageData()
        grid.dimensions = [50, 50, 50]  # adjust grid resolution as needed
        grid.origin = [x_min - padding, y_min - padding, z_min - padding]  # start point of the grid
        grid.spacing = [(x_max - x_min + 2 * padding) / (grid.dimensions[0] - 1),
                        (y_max - y_min + 2 * padding) / (grid.dimensions[1] - 1),
                        (z_max - z_min + 2 * padding) / (grid.dimensions[2] - 1)]  # grid spacing

        # Calculate the distances from each grid point to the nearest point on the surface mesh
        distances = grid.compute_implicit_distance(surface_mesh, inplace=False)

        # Assign these distances as a scalar field
        grid.point_data['Distance'] = distances.point_data['implicit_distance']
        scalar = np.array(grid.point_data['Distance'])
        scalar_field = scalar.reshape(50, 50, 50)
        return scalar_field

    def plot_scalar_field_in_gui(self, scalar_field):
        def plot():
            plotter = pv.Plotter(notebook=False)
            surface_mesh = self.mesh.extract_surface()

            # Create a uniform grid surrounding the mesh
            x_min, x_max, y_min, y_max, z_min, z_max = surface_mesh.bounds
            padding = 10  # add some padding to the grid dimensions
            grid = pv.ImageData()
            grid.dimensions = [50, 50, 50]  # adjust grid resolution as needed
            grid.origin = [x_min - padding, y_min - padding, z_min - padding]  # start point of the grid
            grid.spacing = [(x_max - x_min + 2 * padding) / (grid.dimensions[0] - 1),
                            (y_max - y_min + 2 * padding) / (grid.dimensions[1] - 1),
                            (z_max - z_min + 2 * padding) / (grid.dimensions[2] - 1)]  # grid spacing

            # Calculate the distances from each grid point to the nearest point on the surface mesh
            distances = grid.compute_implicit_distance(surface_mesh, inplace=False)

            # Assign these distances as a scalar field
            grid.point_data['Distance'] = distances.point_data['implicit_distance']

            plotter.add_mesh(surface_mesh, color='white', label='Mesh Surface')
            plotter.add_mesh(grid, scalars='Distance', cmap='viridis', opacity=0.7, label='Scalar Field')
            plotter.add_legend()
            plotter.show()

        plot_thread = threading.Thread(target=plot)
        plot_thread.start()

    def save_scalar_field(self):
        file_path = self.scalar_file_path.get()
        if file_path:
            np.save(file_path,self.vtk_to_scalar(self.mesh))
            messagebox.showinfo("Save Scalar Field", "Scalar field saved successfully!")

    def create_model(self):
        geo_model = self._create_model_logic(self.auto_corr.get())
        sol = gp.compute_model(geo_model)
        self._plot_2d(geo_model)

    def plot_3d(self):
        geo_model = self._create_model_logic(self.auto_corr.get())
        sol = gp.compute_model(geo_model)
        self._plot_3d(geo_model)

    def _create_model_logic(self, auto_corr):
        # Initialize a list to store orientation data for each slice
        orientation_data = []
        # Iterate over slices
        self.x_points = np.array(self.x_points, dtype=object)
        self.y_points = np.array(self.y_points, dtype=object)

        for n in range(np.array(self.x_points).shape[0]):
            slice_x = np.array(self.x_points)[n]
            slice_y = np.array(self.y_points)[n]

            # Calculate the max points in x and y slices
            slice_x_max = np.array([slice_x[:, 0][slice_x[:, 1] == slice_x[:, 1].min()][0], slice_x[:, 1].min()])
            slice_y_max = np.array([slice_y[:, 0][slice_y[:, 1] == slice_y[:, 1].min()][0], slice_y[:, 1].min()])

            # Apply correction if auto_corr is True
            if auto_corr:
                diff = slice_x_max - slice_y_max
                slice_y[:, 1] += diff[1]

            # Generate pairs of indices for slice_x and slice_y
            array_x = list(range(slice_x.shape[0]))
            result_x = [(array_x[i], array_x[i + 1]) for i in range(len(array_x) - 1)]
            array_y = list(range(slice_y.shape[0]))
            result_y = [(array_y[i], array_y[i + 1]) for i in range(len(array_y) - 1)]

            # Calculate orientations for slice_x
            for i in result_x:
                # Calculate the pole vector
                pole_x = [self.cal_orientation(slice_x[i[0]], slice_x[i[1]])[0], 0,
                self.cal_orientation(slice_x[i[0]], slice_x[i[1]])[1]]
                    
                # Append orientation data as a dictionary
                orientation_data.append({
                'x': slice_x[i[0]][0],
                'y': int(self.x_pos_saved[n]),
                'z': -slice_x[i[0]][1],
                'surface': self.name_saved[n],
                'G_x': pole_x[0],
                'G_y': pole_x[1],
                    'G_z': pole_x[2]
                    })

            # Calculate orientations for slice_y
            for i in result_y:
                # Calculate the pole vector
                pole_y = [0, self.cal_orientation(slice_y[i[0]], slice_y[i[1]])[0],
                        self.cal_orientation(slice_y[i[0]], slice_y[i[1]])[1]]
                
                # Append orientation data as a dictionary
                orientation_data.append({
                    'x': int(self.y_pos_saved[n]),
                    'y': slice_y[i[0]][0],
                    'z': -slice_y[i[0]][1],
                    'surface': self.name_saved[n],
                    'G_x': pole_y[0],
                    'G_y': pole_y[1],
                    'G_z': pole_y[2]
                })

        # Create DataFrame from orientation data
        orientations_df = pd.DataFrame(orientation_data)
        orientations_df.to_csv("Orientations.csv")
        surface_points = self.surface_all(auto_corr)
        surface_points.to_csv("Surface_points.csv")
        geo_model: gp.data.GeoModel = gp.create_geomodel(
        project_name='Salt',
        extent=[0, 800, 0, 800, -600, 0],
        resolution = [50, 50, 50],  # * Here we define the number of octree levels. If octree levels are defined, the resolution is ignored.
        importer_helper=gp.data.ImporterHelper(
        path_to_orientations="Orientations.csv",
        path_to_surface_points="Surface_points.csv",
        )
        )

        return geo_model

    def _plot_2d(self, geo_model):
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))

        for ax in gpv.plot_2d(geo_model, direction='x', show_data=True).axes:
            fig_x = ax.get_figure()
            fig_x.canvas.draw()
            axes[0].imshow(np.array(fig_x.canvas.renderer.buffer_rgba()))
            axes[0].axis('off')

        for ax in gpv.plot_2d(geo_model, direction='y', show_data=True).axes:
            fig_y = ax.get_figure()
            fig_y.canvas.draw()
            axes[1].imshow(np.array(fig_y.canvas.renderer.buffer_rgba()))
            axes[1].axis('off')

        canvas = FigureCanvasTkAgg(fig, master=self.plot_2d_frame)
        canvas.draw()
        canvas.get_tk_widget().pack()

    def _plot_3d(self, geo_model):
        frame = tk.Frame(self.plot_2d_frame, width=800, height=600)
        frame.pack(padx=10, pady=10)
        gpv_3d = gpv.plot_3d(model=geo_model,show_surfaces=False,show_data=True,show_lith=False,image=False)

    # Methods for the RBF Interpolation Tab
    def load_surface_points(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.vertices_df = pd.read_excel(file_path)
            messagebox.showinfo("RBF Interpolation", "Surface points loaded successfully!")
        self.model_name = file_path.split('/')[-1].split('_')[0]

    def load_orientation_points(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.normals_df = pd.read_excel(file_path)
            messagebox.showinfo("RBF Interpolation", "Orientation points loaded successfully!")

    def load_orientations(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.normals_o_df = pd.read_excel(file_path)
            messagebox.showinfo("RBF Interpolation", "Orientations loaded successfully!")

    def load_cv_surface_points(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.cv_vertices_df = pd.read_excel(file_path)
            messagebox.showinfo("RBF Interpolation", "Surface points loaded successfully!")

    def load_cv_orientation_points(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            self.cv_normals_df = pd.read_excel(file_path)
            messagebox.showinfo("RBF Interpolation", "Orientation points loaded successfully!")

    def create_cross_validation_tab(self):
            # Tab 7: Cross-Validation
        self.cv_frame = tk.Frame(self.tab7)
        self.cv_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_surface_cv_button = ttk.Button(self.cv_frame, text="Load Surface Points", style='TButton', command=self.load_cv_surface_points)
        self.load_surface_cv_button.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_orientation_cv_button = ttk.Button(self.cv_frame, text="Load Orientation Points", style='TButton', command=self.load_cv_orientation_points)
        self.load_orientation_cv_button.grid(row=0, column=1, padx=10, pady=10, sticky='w')

        tk.Label(self.cv_frame, text="Sampling method:", font=self.label_font).grid(row=0, column=2, padx=10, pady=5, sticky='w')
        self.Sampling_method_cv = ttk.Combobox(self.cv_frame, font=self.entry_font, values=[
            'Random','Systematic' ,'Statistic(K-means)',])
        self.Sampling_method_cv.grid(row=0, column=3, padx=10, pady=5, sticky='w')

        tk.Label(self.cv_frame, text="Number of Sample Surface Points:", font=self.label_font).grid(row=1, column=0, padx=10, pady=5, sticky='w')
        self.num_surface_points_cv = tk.Entry(self.cv_frame, font=self.entry_font)
        self.num_surface_points_cv.grid(row=1, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.cv_frame, text="Number of Sample Orientation Points:", font=self.label_font).grid(row=2, column=0, padx=10, pady=5, sticky='w')
        self.num_orientation_points_cv = tk.Entry(self.cv_frame, font=self.entry_font)
        self.num_orientation_points_cv.grid(row=2, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.cv_frame, text="Kernel Function:", font=self.label_font).grid(row=3, column=0, padx=10, pady=5, sticky='w')
        self.kernel_function_cv = ttk.Combobox(self.cv_frame, font=self.entry_font, values=[
            'cubic', 'gaussian', 'quintic', 'cubic_covariance', 'multiquadric', 'linear', 
            'thin_plate', 'inverse_quadratic', 'inverse_multiquadric', 'gaussian_covariance', 
            'ga_cub_cov', 'ga_cub'])
        self.kernel_function_cv.grid(row=3, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.cv_frame, text="Drift Type:", font=self.label_font).grid(row=4, column=0, padx=10, pady=5, sticky='w')
        self.drift_type_cv = ttk.Combobox(self.cv_frame, font=self.entry_font, values=[
            'None', 'First Order Polynomial','Second Order Polynomial', 'Ellipsoid', 'Dome Shaped'])
        self.drift_type_cv.grid(row=4, column=1, padx=10, pady=5, sticky='w')

        tk.Label(self.cv_frame, text="eps range (start, stop, step):", font=self.label_font).grid(row=5, column=0, padx=10, pady=5, sticky='w')
        self.epsilon_start_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.epsilon_start_cv.place(relx=0.63, rely=0.36, anchor='center')
        self.epsilon_stop_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.epsilon_stop_cv.place(relx=0.73, rely=0.36, anchor='center')
        self.epsilon_step_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.epsilon_step_cv.place(relx=0.83, rely=0.36, anchor='center')

        tk.Label(self.cv_frame, text="Range value range (start, stop, step):", font=self.label_font).grid(row=6, column=0, padx=10, pady=5, sticky='w')
        self.range_start_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.range_start_cv.place(relx=0.63, rely=0.42, anchor='center')
        self.range_stop_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.range_stop_cv.place(relx=0.73, rely=0.42, anchor='center')
        self.range_step_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.range_step_cv.place(relx=0.83, rely=0.42, anchor='center')

        tk.Label(self.cv_frame, text="Parameter a range (start, stop, step):", font=self.label_font).grid(row=7, column=0, padx=10, pady=5, sticky='w')
        self.param_a_start_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_a_start_cv.place(relx=0.63, rely=0.48, anchor='center')
        self.param_a_stop_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_a_stop_cv.place(relx=0.73, rely=0.48, anchor='center')
        self.param_a_step_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_a_step_cv.place(relx=0.83, rely=0.48, anchor='center')

        tk.Label(self.cv_frame, text="Parameter b range (start, stop, step):", font=self.label_font).grid(row=8, column=0, padx=10, pady=5, sticky='w')
        self.param_b_start_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_b_start_cv.place(relx=0.63, rely=0.54, anchor='center')
        self.param_b_stop_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_b_stop_cv.place(relx=0.73, rely=0.54, anchor='center')
        self.param_b_step_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_b_step_cv.place(relx=0.83, rely=0.54, anchor='center')

        tk.Label(self.cv_frame, text="Parameter c range (start, stop, step):", font=self.label_font).grid(row=9, column=0, padx=10, pady=5, sticky='w')
        self.param_c_start_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_c_start_cv.place(relx=0.63, rely=0.60, anchor='center')
        self.param_c_stop_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_c_stop_cv.place(relx=0.73, rely=0.60, anchor='center')
        self.param_c_step_cv = tk.Entry(self.cv_frame, font=self.entry_font, width=5)
        self.param_c_step_cv.place(relx=0.83, rely=0.60, anchor='center')

        self.compute_cv_button = ttk.Button(self.cv_frame, text="Compute Cross-Validation", style='TButton', command=self.run_cross_validation)
        self.compute_cv_button.grid(row=10, column=0, padx=10, pady=10, sticky='w')

        self.progress = ttk.Progressbar(self.cv_frame, orient=tk.HORIZONTAL, length=300, mode='determinate')
        self.progress.grid(row=12, column=0, columnspan=4, padx=10, pady=10, sticky='w')

        self.elapsed_time_label = tk.Label(self.cv_frame, text="", font=self.label_font)
        self.elapsed_time_label.grid(row=13, column=0, columnspan=4, padx=10, pady=10, sticky='w')

        self.remaining_time_label = tk.Label(self.cv_frame, text="", font=self.label_font)
        self.remaining_time_label.grid(row=14, column=0, columnspan=4, padx=10, pady=10, sticky='w')

        self.result_label = tk.Label(self.cv_frame, text="", font=self.label_font)
        self.result_label.grid(row=11, column=0, columnspan=4, padx=10, pady=10, sticky='w')

    def create_scalar_comparison_tab(self):
        # Tab 7: Scalar Field Comparison
        self.scalar_comparison_frame = tk.Frame(self.tab8)
        self.scalar_comparison_frame.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_scalar1_button = ttk.Button(self.scalar_comparison_frame, text="Load Scalar Field 1", style='TButton', command=self.load_scalar1)
        self.load_scalar1_button.grid(row=0, column=0, padx=10, pady=10, sticky='w')

        self.load_scalar2_button = ttk.Button(self.scalar_comparison_frame, text="Load Scalar Field 2", style='TButton', command=self.load_scalar2)
        self.load_scalar2_button.grid(row=0, column=1, padx=10, pady=10, sticky='w')

        self.load_scalar3_button = ttk.Button(self.scalar_comparison_frame, text="Load Scalar Field 3", style='TButton', command=self.load_scalar3)
        self.load_scalar3_button.grid(row=0, column=2, padx=10, pady=10, sticky='w')

        self.load_scalar4_button = ttk.Button(self.scalar_comparison_frame, text="Load Scalar Field 4", style='TButton', command=self.load_scalar4)
        self.load_scalar4_button.grid(row=0, column=3, padx=10, pady=10, sticky='w')

        self.load_scalar5_button = ttk.Button(self.scalar_comparison_frame, text="Load Scalar Field 5", style='TButton', command=self.load_scalar5)
        self.load_scalar5_button.grid(row=0, column=4, padx=10, pady=10, sticky='w')

        tk.Label(self.scalar_comparison_frame, text="Compare Plane:", font=self.label_font).grid(row=1, column=0, padx=10, pady=5, sticky='w')
        self.compare_plane = ttk.Combobox(self.scalar_comparison_frame, font=self.entry_font, values=['x', 'y', 'z'])
        self.compare_plane.grid(row=1, column=1, padx=10, pady=5, sticky='w')
        self.compare_plane.current(1)  

        self.flip_checkbox = tk.Checkbutton(self.scalar_comparison_frame, text="Flip Scalar Field 1", variable=self.use_flip, font=self.label_font)
        self.flip_checkbox.grid(row=2, column=0, padx=10, pady=5, sticky='w')

        self.reverse_checkbox = tk.Checkbutton(self.scalar_comparison_frame, text="Reverse Scalar Field 1", variable=self.use_reverse, font=self.label_font)
        self.reverse_checkbox.grid(row=2, column=1, padx=10, pady=5, sticky='w')

        self.compare_button = ttk.Button(self.scalar_comparison_frame, text="Compare", style='TButton', command=self.compare_scalars)
        self.compare_button.grid(row=3, column=0, padx=10, pady=10, sticky='w')

        self.scalar_canvas_frame = tk.Frame(self.tab8)
        self.scalar_canvas_frame.grid(row=4, column=0, padx=10, pady=10, columnspan=4)


    def compare_scalars(self):
        if self.scalar1 is None or self.scalar2 is None:
            messagebox.showerror("Error", "Please load both scalar fields before comparison.")
            return

        scalar1 = self.scalar1
        scalar2 = self.scalar2

        scalar3 = getattr(self, 'scalar3', None)
        scalar4 = getattr(self, 'scalar4', None)
        scalar5 = getattr(self, 'scalar5', None)

        flipped_field1 = np.zeros_like(scalar1)
        if self.use_flip.get():
            for i in range(scalar1.shape[1]):  
                flipped_field1[:, i, :] = np.fliplr(scalar1[:, i, :])
                scalar1 = flipped_field1

        if self.use_reverse.get():
            scalar1 = -scalar1

        axis = self.compare_plane.get()
        mid_index = {
            'x': scalar1.shape[0] // 2,
            'y': scalar1.shape[1] // 2,
            'z': scalar1.shape[2] // 2
        }[axis]

        self.scalar_comparison(scalar1, scalar2,scalar3, scalar4,scalar5, axis, mid_index)

    def scalar_comparison(self, scalar1, scalar2,scalar3=None, scalar4=None,scalar5=None, axis=None, index=None):
        def plot_two_gradient_fields(field1, field2, diff, axis, index, title, skip=5):
            grad_field1 = np.gradient(field1)
            grad_field2 = np.gradient(field2)
            if axis == 'x':
                grad_slice1 = [grad_field1[0][index, :, :], grad_field1[1][index, :, :], grad_field1[2][index, :, :]]
                grad_slice2 = [grad_field2[0][index, :, :], grad_field2[1][index, :, :], grad_field2[2][index, :, :]]
                diff_slice = diff[index, :, :]
            elif axis == 'y':
                grad_slice1 = [grad_field1[0][:, index, :], grad_field1[1][:, index, :], grad_field1[2][:, index, :]]
                grad_slice2 = [grad_field2[0][:, index, :], grad_field2[1][:, index, :], grad_field2[2][:, index, :]]
                diff_slice = diff[:, index, :]
            elif axis == 'z':
                grad_slice1 = [grad_field1[0][:, :, index], grad_field1[1][:, :, index], grad_field1[2][:, :, index]]
                grad_slice2 = [grad_field2[0][:, :, index], grad_field2[1][:, :, index], grad_field2[2][:, :, index]]
                diff_slice = diff[:, :, index]
            else:
                raise ValueError("Axis must be 'x', 'y', or 'z'")

            fig, ax = plt.subplots(figsize=(3, 2.5))  

            grad_x_1 = grad_slice1[0][::skip, ::skip]
            grad_y_1 = grad_slice1[1][::skip, ::skip]
            magnitude1 = np.sqrt(grad_x_1**2 + grad_y_1**2)

            grad_x_2 = grad_slice2[0][::skip, ::skip]
            grad_y_2 = grad_slice2[1][::skip, ::skip]
            magnitude2 = np.sqrt(grad_x_2**2 + grad_y_2**2)

            epsilon = 1e-10
            grad_x_normalized_1 = grad_x_1 / (magnitude1 + epsilon)
            grad_y_normalized_1 = grad_y_1 / (magnitude1 + epsilon)
            grad_x_normalized_2 = grad_x_2 / (magnitude2 + epsilon)
            grad_y_normalized_2 = grad_y_2 / (magnitude2 + epsilon)
            fixed_arrow_length = 6



            contour = ax.contourf(diff_slice, levels=30, cmap='cmc.grayC_r')
            X, Y = np.meshgrid(np.arange(0, field1.shape[1], skip), np.arange(0, field1.shape[2], skip))
            ax.quiver(X, Y, grad_x_normalized_1 * fixed_arrow_length, grad_y_normalized_1 * fixed_arrow_length,
                        angles='xy', scale_units='xy', scale=1, color="orange", width=0.003, label='Synthetic Model')
            ax.quiver(X, Y, grad_x_normalized_2 * fixed_arrow_length, grad_y_normalized_2 * fixed_arrow_length,
                        angles='xy', scale_units='xy', scale=1, color="blue", width=0.003, label='Interpolated Model')
            legend = plt.legend(frameon=True,fontsize=8)
            frame = legend.get_frame()
            frame.set_edgecolor('black')
            ax.set_title(f'{title} (Slice {axis}={index})', fontsize=8)
            ax.set_xticklabels([])  
            ax.set_yticklabels([]) 

            return fig

        def plot_scalar_field_slice(field, axis, index, title):
            if axis == 'x':
                slice_data = field[index, :, :]
            elif axis == 'y':
                slice_data = field[:, index, :]
            elif axis == 'z':
                slice_data = field[:, :, index]
            else:
                raise ValueError("Axis must be 'x', 'y', or 'z'")

            fig, ax = plt.subplots(figsize=(3, 2.5))  
            contourf = ax.contourf(slice_data, levels=20, cmap='cmc.batlow')
            contour = ax.contour(slice_data, levels=[float(0)], cmap='gray')
            ax.set_title(f'{title} (Slice {axis}={index})', fontsize=8)
            ax.set_xticklabels([]) 
            ax.set_yticklabels([])  
            return fig

        f_A = scalar1
        f_B = scalar2
        f_C = scalar3 if scalar3 is not None else None
        f_D = scalar4 if scalar4 is not None else None
        f_E = scalar5 if scalar5 is not None else None

        grad_f_A = np.gradient(f_A)
        grad_f_B = np.gradient(f_B)
        grad_f_C = np.gradient(f_C) if f_C is not None else None
        grad_f_D = np.gradient(f_D) if f_D is not None else None
        grad_f_E = np.gradient(f_E) if f_E is not None else None

        dot_product_AB = np.zeros(f_A.shape)
        dot_product_AC = np.zeros(f_A.shape) if f_C is not None else None
        dot_product_AD = np.zeros(f_A.shape) if f_D is not None else None
        dot_product_AE = np.zeros(f_A.shape) if f_E is not None else None
        magnitude_A = np.zeros(f_A.shape)
        magnitude_B = np.zeros(f_A.shape)
        magnitude_C = np.zeros(f_A.shape) if f_C is not None else None
        magnitude_D = np.zeros(f_A.shape) if f_D is not None else None
        magnitude_E = np.zeros(f_A.shape) if f_E is not None else None

        for i in range(3):
            dot_product_AB += grad_f_A[i] * grad_f_B[i]
            magnitude_A += grad_f_A[i] ** 2
            magnitude_B += grad_f_B[i] ** 2
            if f_C is not None:
                dot_product_AC += grad_f_A[i] * grad_f_C[i]
                magnitude_C += grad_f_C[i] ** 2 
            if f_D is not None:
                dot_product_AD += grad_f_A[i] * grad_f_D[i]
                magnitude_D += grad_f_D[i] ** 2
            if f_E is not None:
                dot_product_AE += grad_f_A[i] * grad_f_E[i]
                magnitude_E += grad_f_E[i] ** 2


        magnitude_A = np.sqrt(magnitude_A)
        magnitude_B = np.sqrt(magnitude_B)
        magnitude_C = np.sqrt(magnitude_C) if f_C is not None else None
        magnitude_D = np.sqrt(magnitude_D) if f_D is not None else None
        magnitude_E = np.sqrt(magnitude_E) if f_E is not None else None

        epsilon = 1e-10
        magnitude_A = np.where(magnitude_A == 0, epsilon, magnitude_A)
        magnitude_B = np.where(magnitude_B == 0, epsilon, magnitude_B)
        cosine_similarity_AB = dot_product_AB / (magnitude_A * magnitude_B)
        d_AB_nabla = 1 - cosine_similarity_AB
        if f_C is not None:
            magnitude_C = np.where(magnitude_C == 0, epsilon, magnitude_C)
            cosine_similarity_AC = dot_product_AC / (magnitude_A * magnitude_C)
            d_AC_nabla = 1 - cosine_similarity_AC
        if f_D is not None:
            magnitude_D = np.where(magnitude_D == 0, epsilon, magnitude_D)
            cosine_similarity_AD = dot_product_AD / (magnitude_A * magnitude_D)
            d_AD_nabla = 1 - cosine_similarity_AD
        if f_E is not None:
            magnitude_E = np.where(magnitude_E == 0, epsilon, magnitude_E)
            cosine_similarity_AE = dot_product_AE / (magnitude_A * magnitude_E)
            d_AE_nabla = 1 - cosine_similarity_AE

        figA = plot_scalar_field_slice(f_A, axis, index, 'Synthetic Scalar Field', )
        figB = plot_scalar_field_slice(f_B, axis, index, 'Interpolated Scalar Field', ) 
        figAB = plot_two_gradient_fields(f_A, f_B, d_AB_nabla, axis, index, 'Difference plot', skip=7)
        if f_C is not None:
            figC = plot_scalar_field_slice(f_C, axis, index, 'Interpolated Scalar Field', )
            figAC = plot_two_gradient_fields(f_A, f_C, d_AC_nabla, axis, index, 'Difference plot', skip=7)
        if f_D is not None:
            figD = plot_scalar_field_slice(f_D, axis, index, 'Interpolated Scalar Field', )
            figAD = plot_two_gradient_fields(f_A, f_D, d_AD_nabla, axis, index, 'Difference plot', skip=7)
        if f_E is not None:
            figE = plot_scalar_field_slice(f_E, axis, index, 'Interpolated Scalar Field', )                                      
            figAE = plot_two_gradient_fields(f_A, f_E, d_AE_nabla, axis, index, 'Difference plot', skip=7)

        mean_diff_AB = np.mean(d_AB_nabla)
        rms_diff_AB = np.sqrt(np.mean(d_AB_nabla**2))
        max_diff_AB = np.max(np.abs(d_AB_nabla))
        median_diff_AB = np.median(d_AB_nabla)

        if f_C is not None:
            mean_diff_AC = np.mean(d_AC_nabla)
            rms_diff_AC = np.sqrt(np.mean(d_AC_nabla**2))
            max_diff_AC = np.max(np.abs(d_AC_nabla))
            median_diff_AC = np.median(d_AC_nabla)
        if f_D is not None:
            mean_diff_AD = np.mean(d_AD_nabla)
            rms_diff_AD = np.sqrt(np.mean(d_AD_nabla**2))
            max_diff_AD = np.max(np.abs(d_AD_nabla))
            median_diff_AD = np.median(d_AD_nabla)
        if f_E is not None:
            mean_diff_AE = np.mean(d_AE_nabla)
            rms_diff_AE = np.sqrt(np.mean(d_AE_nabla**2))
            max_diff_AE = np.max(np.abs(d_AE_nabla))
            median_diff_AE = np.median(d_AE_nabla)

        for widget in self.scalar_canvas_frame.winfo_children():
            widget.destroy()

        stats_frame = tk.Frame(self.scalar_canvas_frame)
        stats_frame.grid(row=0, column=1, columnspan=6, pady=5)

        tk.Label(stats_frame, text=f"Mean Difference: {mean_diff_AB:.3f}", font=self.label_font_s).grid(row=0, column=2, sticky='w')
        tk.Label(stats_frame, text=f"RMS Difference: {rms_diff_AB:.3f}", font=self.label_font_s).grid(row=0, column=3, sticky='w')
        tk.Label(stats_frame, text=f"Maximum Difference: {max_diff_AB:.3f}", font=self.label_font_s).grid(row=1, column=2, sticky='w')
        tk.Label(stats_frame, text=f"Median Difference: {median_diff_AB:.3f}", font=self.label_font_s).grid(row=1, column=3, sticky='w')

        if f_C is not None:
            tk.Label(stats_frame, text=f"Mean Difference: {mean_diff_AC:.3f}", font=self.label_font_s).grid(row=0, column=4, sticky='w')
            tk.Label(stats_frame, text=f"RMS Difference: {rms_diff_AC:.3f}", font=self.label_font_s).grid(row=0, column=5, sticky='w')
            tk.Label(stats_frame, text=f"Maximum Difference: {max_diff_AC:.3f}", font=self.label_font_s).grid(row=1, column=4, sticky='w')
            tk.Label(stats_frame, text=f"Median Difference: {median_diff_AC:.3f}", font=self.label_font_s).grid(row=1, column=5, sticky='w')
        
        if f_D is not None:
            tk.Label(stats_frame, text=f"Mean Difference: {mean_diff_AD:.3f}", font=self.label_font_s).grid(row=0, column=6, sticky='w')
            tk.Label(stats_frame, text=f"RMS Difference: {rms_diff_AD:.3f}", font=self.label_font_s).grid(row=0, column=7, sticky='w')
            tk.Label(stats_frame, text=f"Maximum Difference: {max_diff_AD:.3f}", font=self.label_font_s).grid(row=1, column=6, sticky='w')
            tk.Label(stats_frame, text=f"Median Difference: {median_diff_AD:.3f}", font=self.label_font_s).grid(row=1, column=7, sticky='w')

        if f_E is not None:
            tk.Label(stats_frame, text=f"Mean Difference: {mean_diff_AE:.3f}", font=self.label_font_s).grid(row=0, column=8, sticky='w')
            tk.Label(stats_frame, text=f"RMS Difference: {rms_diff_AE:.3f}", font=self.label_font_s).grid(row=0, column=9, sticky='w')
            tk.Label(stats_frame, text=f"Maximum Difference: {max_diff_AE:.3f}", font=self.label_font_s).grid(row=1, column=8, sticky='w')
            tk.Label(stats_frame, text=f"Median Difference: {median_diff_AE:.3f}", font=self.label_font_s).grid(row=1, column=9, sticky='w')

        canvas1 = FigureCanvasTkAgg(figA, master=self.scalar_canvas_frame)
        canvas1.draw()
        canvas1.get_tk_widget().grid(row=1, column=0, padx=5, pady=5)

        canvas2 = FigureCanvasTkAgg(figB, master=self.scalar_canvas_frame)
        canvas2.draw()
        canvas2.get_tk_widget().grid(row=1, column=1, padx=5, pady=5)

        canvas6 = FigureCanvasTkAgg(figAB, master=self.scalar_canvas_frame)
        canvas6.draw()
        canvas6.get_tk_widget().grid(row=2, column=1, padx=5, pady=5)

        if f_C is not None:
            canvas3 = FigureCanvasTkAgg(figC, master=self.scalar_canvas_frame)
            canvas3.draw()
            canvas3.get_tk_widget().grid(row=1, column=2, padx=5, pady=5)
            canvas7 = FigureCanvasTkAgg(figAC, master=self.scalar_canvas_frame)
            canvas7.draw()
            canvas7.get_tk_widget().grid(row=2, column=2, padx=5, pady=5)
        
        if f_D is not None:
            canvas4 = FigureCanvasTkAgg(figD, master=self.scalar_canvas_frame)
            canvas4.draw()
            canvas4.get_tk_widget().grid(row=1, column=3, padx=5, pady=5)
            canvas8 = FigureCanvasTkAgg(figAD, master=self.scalar_canvas_frame)
            canvas8.draw()
            canvas8.get_tk_widget().grid(row=2, column=3, padx=5, pady=5)

        if f_E is not None:
            canvas5 = FigureCanvasTkAgg(figE, master=self.scalar_canvas_frame)
            canvas5.draw()
            canvas5.get_tk_widget().grid(row=1, column=4, padx=5, pady=5)
            canvas9 = FigureCanvasTkAgg(figAE, master=self.scalar_canvas_frame)
            canvas9.draw()
            canvas9.get_tk_widget().grid(row=2, column=4, padx=5, pady=5)

    def load_scalar1(self):
        file_path = filedialog.askopenfilename(filetypes=[("NumPy files", "*.npy")])
        if file_path:
            self.scalar1 = np.load(file_path)
            messagebox.showinfo("Load Scalar Field 1", "Scalar Field 1 loaded successfully!")

    def load_scalar2(self):
        file_path = filedialog.askopenfilename(filetypes=[("NumPy files", "*.npy")])
        if file_path:
            self.scalar2 = np.load(file_path)
            messagebox.showinfo("Load Scalar Field 2", "Scalar Field 2 loaded successfully!")

    def load_scalar3(self):
        file_path = filedialog.askopenfilename(filetypes=[("NumPy files", "*.npy")])
        if file_path:
            self.scalar3 = np.load(file_path)
            messagebox.showinfo("Load Scalar Field 3", "Scalar Field 3 loaded successfully!")

    def load_scalar4(self):
        file_path = filedialog.askopenfilename(filetypes=[("NumPy files", "*.npy")])
        if file_path:
            self.scalar4 = np.load(file_path)
            messagebox.showinfo("Load Scalar Field 4", "Scalar Field 4 loaded successfully!")

    def load_scalar5(self):
        file_path = filedialog.askopenfilename(filetypes=[("NumPy files", "*.npy")])
        if file_path:
            self.scalar5 = np.load(file_path)
            messagebox.showinfo("Load Scalar Field 5", "Scalar Field 5 loaded successfully!")

    def load_drift_scalar(self):
        file_path = filedialog.askopenfilename(filetypes=[("NumPy files", "*.npy")])
        if file_path:
            self.drift_scalar = np.load(file_path)
            messagebox.showinfo("Load Drift Scalar Field", "Drift Scalar Field loaded successfully!")

    def RBF_3D_kernel(self, G_position, G_orientation, layer_position, test_data, eps, range_val, drift=0, cv=True, a=1, b=1, c=1):

        seed = 55
        num_surface_points = int(self.num_surface_points_cv.get())
        num_orientation_points = int(self.num_orientation_points_cv.get())
        func = self.kernel_function_cv.get()
        G_1 = G_position
        G_1_o = G_orientation
        layer1 = layer_position

        def squared_euclidean_distance(x_1, x_2):
            sqd = np.sqrt(np.reshape(np.sum(x_1**2, 1), newshape=(x_1.shape[0], 1)) +
                        np.reshape(np.sum(x_2**2, 1), newshape=(1, x_2.shape[0])) -
                        2 * (x_1 @ x_2.T))
            return np.nan_to_num(sqd)
        dis = squared_euclidean_distance(self.systematic_sampling(self.cv_vertices_df, num_surface_points)[['X','Y','Z']].values, self.systematic_sampling(self.cv_vertices_df, num_surface_points)[['X','Y','Z']].values)
        _eps = dis.mean()
        _range_val = dis.max() * 2

        def kernel(radius, function, a=1, b=1, n=1):
            if function == 'cubic':
                return radius**3
            elif function == 'gaussian':
                return np.exp(-1 * (radius**2)/ (2* eps**2))
            elif function == 'quintic':
                return radius**5
            elif function == 'cubic_covariance':
                return (1 - 7 * (radius / range_val) ** 2 +
                        35 / 4 * (radius / range_val) ** 3 -
                        7 / 2 * (radius / range_val) ** 5 +
                        3 / 4 * (radius / range_val) ** 7)
            elif function == 'multiquadric':
                return np.sqrt(radius**2+eps**2)
            elif function == 'linear':
                return -radius
            elif function == 'thin_plate':
                return radius**2 * np.log(radius)
            elif function == 'inverse_quadratic':
                return 1 / (eps**2 + radius**2)
            elif function == 'inverse_multiquadric':
                return 1 / np.sqrt(radius**2+eps**2)
            elif function == 'gaussian_covariance':
                return np.exp(-(radius**2 / range_val**2))
            elif function == 'ga_cub_cov':
                return a * (1 - np.exp(-(radius / range_val)**2)) + b * (1 - 7 * (radius / range_val) ** 2 +
                    35 / 4 * (radius / range_val) ** 3 - 7 / 2 * (radius / range_val) ** 5 +
                    3 / 4 * (radius / range_val) ** 7)
            elif function == 'ga_cub':
                return np.exp(-1 * radius**2 / (2 * eps**2)) + n * (radius**3)

        df = egrad(kernel)
        ddf = egrad(df)

        def squared_euclidean_distance(x_1, x_2):
            sqd = np.sqrt(np.reshape(np.sum(x_1**2, 1), newshape=(x_1.shape[0], 1)) +
                        np.reshape(np.sum(x_2**2, 1), newshape=(1, x_2.shape[0])) -
                        2 * (x_1 @ x_2.T))
            return np.nan_to_num(sqd)

        def cartesian_dist(x_1, x_2):
            return np.concatenate([
                np.tile(x_1[:, 0] - np.reshape(x_2[:, 0], [x_2[:, 0].shape[0], 1]), [1, 3]),
                np.tile(x_1[:, 1] - np.reshape(x_2[:, 1], [x_2[:, 1].shape[0], 1]), [1, 3]),
                np.tile(x_1[:, 2] - np.reshape(x_2[:, 2], [x_2[:, 2].shape[0], 1]), [1, 3])], axis=0)

        def cov_gradients(dist_tiled):
            a = (h_u * h_v)
            b = dist_tiled**2

            t1 = np.divide(a, b, out=np.zeros_like(a), casting='unsafe', where=b!=0)

            t2 = npy.where(dist_tiled < range_val, npy.nan_to_num(ddf(dist_tiled, function=func)) - npy.nan_to_num(df(dist_tiled, function=func)/dist_tiled), 0)
            
            t3 = perpendicularity_matrix * np.where(dist_tiled < range_val, (np.nan_to_num(df(dist_tiled, function=func)/dist_tiled)), 0)
            t4 = 1/3 * np.eye(dist_tiled.shape[0])

            C_G = t1 * t2 - t3 + t4
            return C_G

        def set_rest_ref_matrix(number_of_points_per_surface):
            ref_layer_points = np.repeat(np.stack([layer1[-1]], axis=0), repeats=number_of_points_per_surface-1, axis=0)
            rest_layer_points = np.concatenate([layer1[0:-1]], axis=0)
            return ref_layer_points, rest_layer_points

        def cov_interface(ref_layer_points, rest_layer_points):
            sed_rest_rest = squared_euclidean_distance(rest_layer_points, rest_layer_points)
            sed_ref_rest = squared_euclidean_distance(ref_layer_points, rest_layer_points)
            sed_rest_ref = squared_euclidean_distance(rest_layer_points, ref_layer_points)
            sed_ref_ref = squared_euclidean_distance(ref_layer_points, ref_layer_points)

            C_I = kernel(sed_rest_rest, function=func) - kernel(sed_ref_rest, function=func) - kernel(sed_rest_ref, function=func) + kernel(sed_ref_ref, function=func)
            return C_I

        def cartesian_dist_no_tile(x_1, x_2):
            return np.concatenate([
                np.transpose((x_1[:, 0] - np.reshape(x_2[:, 0], [x_2.shape[0], 1]))),
                np.transpose((x_1[:, 1] - np.reshape(x_2[:, 1], [x_2.shape[0], 1]))),
                np.transpose((x_1[:, 2] - np.reshape(x_2[:, 2], [x_2.shape[0], 1])))
            ], axis=0)

        def cov_interface_gradients(hu_rest, hu_ref):
            C_GI = hu_rest * np.where(sed_dips_rest < range_val, (np.nan_to_num(df(sed_dips_rest, function=func) / sed_dips_rest)), 0) - \
                hu_ref * np.where(sed_dips_ref < range_val, (np.nan_to_num(df(sed_dips_ref, function=func) / sed_dips_ref)), 0)
            return C_GI

        def perpendicularity(G_1):
            a = np.concatenate([np.ones([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]])], axis=1)
            b = np.concatenate([np.zeros([G_1.shape[0], G_1.shape[0]]), np.ones([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]])], axis=1)
            c = np.concatenate([np.zeros([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]]), np.ones([G_1.shape[0], G_1.shape[0]])], axis=1)
            return np.concatenate([a, b, c], axis=0)
        
        
        G_1_tiled = np.tile(G_1, [3, 1])

        h_u = cartesian_dist(G_1, G_1)
        h_v = h_u.T

        perpendicularity_matrix = perpendicularity(G_1)

        dist_tiled = squared_euclidean_distance(G_1_tiled, G_1_tiled)

        dist_tiled = dist_tiled + np.eye(dist_tiled.shape[0])

        C_G = cov_gradients(dist_tiled)

        number_of_points_per_surface = np.array([layer1.shape[0]])

        ref_layer_points, rest_layer_points = set_rest_ref_matrix(number_of_points_per_surface)

        C_I = cov_interface(ref_layer_points, rest_layer_points)

        sed_dips_rest = squared_euclidean_distance(G_1_tiled, rest_layer_points)
        sed_dips_ref = squared_euclidean_distance(G_1_tiled, ref_layer_points)

        hu_rest = cartesian_dist_no_tile(G_1, rest_layer_points)
        hu_ref = cartesian_dist_no_tile(G_1, ref_layer_points)

        C_GI = cov_interface_gradients(hu_rest, hu_ref)
        C_IG = C_GI.T

        x0 = (layer1[:, 0].min() + layer1[:, 0].max()) / 2
        y0 = (layer1[:, 1].min() + layer1[:, 1].max()) / 2
        z0 = (layer1[:, 2].min() + layer1[:, 2].max()) / 2

        xx = np.linspace(layer1[:, 0].min() - 100, layer1[:, 0].max() + 100, 50)
        yy = np.linspace(layer1[:, 1].min() - 100, layer1[:, 1].max() + 100, 50)
        zz = np.linspace(layer1[:, 2].min() - 100, layer1[:, 2].max() + 100, 50)
        XX, YY, ZZ = np.meshgrid(xx, yy, zz)
        X = np.reshape(XX, [-1]).T
        Y = np.reshape(YY, [-1]).T
        Z = np.reshape(ZZ, [-1]).T
        if cv == True:
            grid = test_data
        else:
            grid = np.stack([X, Y, Z], axis=1)

        hu_Simpoints = cartesian_dist_no_tile(G_1, grid)
        sed_dips_SimPoint = squared_euclidean_distance(G_1_tiled, grid)
        sed_rest_SimPoint = squared_euclidean_distance(rest_layer_points, grid)
        sed_ref_SimPoint = squared_euclidean_distance(ref_layer_points, grid)

        if drift == 1:  # 'Second Order Polynomial'
            K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
            n = G_1.shape[0]
            sub_x = np.tile(np.array([[1,0,0]]),[n,1])
            sub_y = np.tile(np.array([[0,1,0]]),[n,1])
            sub_z = np.tile(np.array([[0,0,1]]),[n,1])
            sub_block1 = np.concatenate([sub_x,sub_y,sub_z],0)

            sub_x_2 = np.zeros((n, 3))
            sub_y_2 = np.zeros((n, 3))
            sub_z_2 = np.zeros((n, 3))
            sub_x_2[:, 0] = 2 * G_1[:, 0]
            sub_y_2[:, 1] = 2 * G_1[:, 1]
            sub_z_2[:, 2] = 2 * G_1[:, 2]

            sub_block2 = np.concatenate([sub_x_2, sub_y_2, sub_z_2], 0)

            sub_xy = np.reshape(np.concatenate([G_1[:, 1],G_1[:, 0]], 0), [2 * n, 1])
            sub_xy = np.pad(sub_xy, [[0, n], [0, 0]])

            sub_xz = np.concatenate([np.pad(np.reshape(G_1[:, 2], [n, 1]), [[0, n], [0, 0]]), np.reshape(G_1[:, 0], [n, 1])], 0)

            sub_yz = np.reshape(np.concatenate([G_1[:, 2], G_1[:, 1]], 0), [2 * n, 1])
            sub_yz = np.pad(sub_yz, [[n, 0], [0, 0]])

            sub_block3 = np.concatenate([sub_xy, sub_xz, sub_yz], 1)

            U_G = np.concatenate([sub_block1, sub_block2, sub_block3], 1)
            # print(U_G.shape)
            U_I = -np.stack([(rest_layer_points[:, 0] - ref_layer_points[:, 0]), 
                            (rest_layer_points[:, 1] - ref_layer_points[:, 1]),
                            (rest_layer_points[:, 2] - ref_layer_points[:, 2]),
                            (rest_layer_points[:, 0] ** 2 - ref_layer_points[:, 0] ** 2),
                            (rest_layer_points[:, 1] ** 2 - ref_layer_points[:, 1] ** 2),
                            (rest_layer_points[:, 2] ** 2 - ref_layer_points[:, 2] ** 2),
                            (rest_layer_points[:, 0] * rest_layer_points[:, 1] - ref_layer_points[:, 0] * ref_layer_points[:, 1]),
                            (rest_layer_points[:, 0] * rest_layer_points[:, 2] - ref_layer_points[:, 0] * ref_layer_points[:, 2]),
                            (rest_layer_points[:, 1] * rest_layer_points[:, 2] - ref_layer_points[:, 1] * ref_layer_points[:, 2])], 1)

            length_of_CG = C_G.shape[1]
            length_of_CGI = C_GI.shape[1]
            U_G = U_G[:length_of_CG, :9]
            U_I = U_I[:length_of_CGI, :9]


            U = np.concatenate([U_G,U_I],0)

            # build zero matrix
            zero_matrix = np.zeros([U.shape[1],U.shape[1]])

            # concatenate drift matrix and kriging matrix
            K_D = np.concatenate([np.concatenate([K,U],axis = 1),np.concatenate([U.T,zero_matrix],axis = 1)],axis = 0)
            # build right side matrix of cokriging system
            bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
            bk = np.reshape(bk,newshape = [bk.shape[0],1])
            # solve kriging weight
            w = np.linalg.lstsq(K_D,bk)[0]
            # gradient contribution
            sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))
            sigma_0_grad = np.sum(sigma_0_grad,axis=0)
            # surface point contribution
            sigma_0_interf = -w[G_1.shape[0]*3:-9]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
            sigma_0_interf = np.sum(sigma_0_interf,axis = 0)
            # 2nd order drift contribution
            sigma_0_2nd_drift_1 = grid[:,0] * w[-9] + grid[:,1] * w[-8] + grid[:,2] * w[-7] + \
                grid[:,0] ** 2 * w[-6] + grid[:,1] ** 2 * w[-5] + grid[:,2] ** 2 * w[-4] + \
                grid[:,0] * grid[:,1] * w[-3] + grid[:,0] * grid[:,2] * w[-2] + grid[:,1] * grid[:,2] * w[-1]
            sigma_0_2nd_drift = sigma_0_2nd_drift_1
            interpolate_result = 1*sigma_0_grad+1*sigma_0_interf + 1*sigma_0_2nd_drift 

        elif drift == 0:  # 'None'
            K_D = np.nan_to_num(np.concatenate([np.concatenate([C_G, C_GI], axis=1), np.concatenate([C_IG, C_I], axis=1)], axis=0))

            bk = np.concatenate([G_1_o[:, 0], G_1_o[:, 1], G_1_o[:, 2], np.zeros(K_D.shape[0] - G_1.shape[0] * 3)], axis=0)
            bk = np.reshape(bk, newshape=[bk.shape[0], 1])

            w = np.linalg.lstsq(K_D, bk, rcond=None)[0]
            sigma_0_grad = w[:G_1.shape[0] * 3] * (-hu_Simpoints * (sed_dips_SimPoint < range_val) * (np.nan_to_num(df(sed_dips_SimPoint, function=func) / sed_dips_SimPoint)))

            sigma_0_grad = np.sum(sigma_0_grad, axis=0)
            sigma_0_interf = -w[G_1.shape[0] * 3:] * (((sed_rest_SimPoint < range_val) * (kernel(sed_rest_SimPoint, function=func)) - (sed_ref_SimPoint < range_val) * (kernel(sed_ref_SimPoint, function=func))))
            sigma_0_interf = np.sum(sigma_0_interf, axis=0)

            interpolate_result = 1 * sigma_0_grad + 1 * sigma_0_interf

        elif drift == 2:  # 'Spherical'
            r = 1
            K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
            D_Z = ((2*(G_1[:,2]-z0)/c)**2).reshape(G_1.shape[0],1)
            D_X = ((2*(G_1[:,0]-x0)/a)**2).reshape(G_1.shape[0],1)
            D_Y = ((2*(G_1[:,1]-y0)/b)**2).reshape(G_1.shape[0],1)
            D_I = ((((ref_layer_points[:,0]-x0)/a)**2+((ref_layer_points[:,1]-y0)/b)**2+((ref_layer_points[:,2]-z0)/c)**2-r**2) \
                - (((rest_layer_points[:,0]-x0)/a)**2+((rest_layer_points[:,1]-y0)/b)**2+((rest_layer_points[:,2]-z0)/c)**2-r**2)).reshape(-1,1)       
            # build external drift matrix
            D_E = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
            D_E_T = D_E.T
            # D = np.concatenate([D_U,D_E],axis=1)
            D = D_E
            D_T = D.T 
            # build zero matrix
            zero_matrix = np.zeros([D.shape[1],D.shape[1]])
            # concatenate drift matrix and kriging matrix
            K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)

            # build right side matrix of cokriging system
            bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
            bk = np.reshape(bk,newshape = [bk.shape[0],1])
            # solve kriging weight
            w = np.linalg.lstsq(K_D,bk)[0]
            # gradient contribution
            sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))
            sigma_0_grad = np.sum(sigma_0_grad,axis=0)
            # surface point contribution
            sigma_0_interf = -w[G_1.shape[0]*3:-1]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
            sigma_0_interf = np.sum(sigma_0_interf,axis = 0)
            external_drift = (((grid[:,0]-x0)/a)**2+((grid[:,1]-y0)/b)**2+((grid[:,2]-z0)/c)**2+r**2) * (w[-1]).T
            interpolate_result = 1*sigma_0_grad+1*sigma_0_interf   + 1* external_drift


        elif drift == 3:  # 'Dome Shaped'
            K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
            # z - (c - (x**2 + y**2) / a**2) * np.exp(-(x**2 + y**2) / b**2)
            def F_x(x, y, z, a, b, c):
                return z - (c - ((x - x0)**2 + (y - y0)**2) / a**2) * np.exp(-((x - x0)**2 + (y - y0)**2) / b**2)
            
            def partial_F_x(x, y, a, b, c):
                term1 = (2 * (x - x0) * (c - ((x - x0)**2 + (y - y0)**2) / a**2)) / b**2
                term2 = (2 * (x - x0)) / a**2
                return (term1 + term2) * np.exp(-((x - x0)**2 + (y - y0)**2) / b**2)

            def partial_F_y(y, x, a, b, c):
                term1 = (2 * (y - y0) * (c - ((x - x0)**2 + (y - y0)**2) / a**2)) / b**2
                term2 = (2 * (y - y0)) / a**2
                return (term1 + term2) * np.exp(-((x - x0)**2 + (y - y0)**2) / b**2)

            def partial_F_z():
                return np.ones_like(x)
            
            x = G_1[:, 0]
            y = G_1[:, 1]
            z = G_1[:, 2]

            D_Z = partial_F_z().reshape(G_1.shape[0],1)
            D_X = partial_F_x(x, y, a, b, c).reshape(G_1.shape[0],1)
            D_Y = partial_F_y(y, x, a, b, c).reshape(G_1.shape[0],1)
            D_I = (F_x(ref_layer_points[:,0], ref_layer_points[:,1], ref_layer_points[:,2], a, b, c) \
                - F_x(rest_layer_points[:,0], rest_layer_points[:,1], rest_layer_points[:,2], a, b, c)).reshape(-1,1)
            # build external drift matrix
            D = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
            D_T = D.T 
            # build zero matrix
            zero_matrix = np.zeros([D.shape[1],D.shape[1]])
            # concatenate drift matrix and kriging matrix
            K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)
            # build right side matrix of cokriging system
            bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
            bk = np.reshape(bk,newshape = [bk.shape[0],1])
            # solve kriging weight
            w = np.linalg.lstsq(K_D,bk)[0]
            # gradient contribution
            sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))
            sigma_0_grad = np.sum(sigma_0_grad,axis=0)
            # surface point contribution
            sigma_0_interf = -w[G_1.shape[0]*3:-1]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
            sigma_0_interf = np.sum(sigma_0_interf,axis = 0)
            external_drift = F_x(grid[:,0],grid[:,1],grid[:,2],a,b,c) * (w[-1]).T
            interpolate_result = 1*sigma_0_grad+1*sigma_0_interf   + 1* external_drift

        if drift == 4:  # 'First Order Polynomial'
            K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
            n = G_1.shape[0]
            sub_x = np.tile(np.array([[1,0,0]]),[n,1])
            sub_y = np.tile(np.array([[0,1,0]]),[n,1])
            sub_z = np.tile(np.array([[0,0,1]]),[n,1])
            sub_block1 = np.concatenate([sub_x,sub_y,sub_z],0)

            sub_x_2 = np.zeros((n, 3))
            sub_y_2 = np.zeros((n, 3))
            sub_z_2 = np.zeros((n, 3))
            sub_x_2[:, 0] = 2 * G_1[:, 0]
            sub_y_2[:, 1] = 2 * G_1[:, 1]
            sub_z_2[:, 2] = 2 * G_1[:, 2]

            sub_block2 = np.concatenate([sub_x_2, sub_y_2, sub_z_2], 0)

            sub_xy = np.reshape(np.concatenate([G_1[:, 1],G_1[:, 0]], 0), [2 * n, 1])
            sub_xy = np.pad(sub_xy, [[0, n], [0, 0]])

            sub_xz = np.concatenate([np.pad(np.reshape(G_1[:, 2], [n, 1]), [[0, n], [0, 0]]), np.reshape(G_1[:, 0], [n, 1])], 0)

            sub_yz = np.reshape(np.concatenate([G_1[:, 2], G_1[:, 1]], 0), [2 * n, 1])
            sub_yz = np.pad(sub_yz, [[n, 0], [0, 0]])

            sub_block3 = np.concatenate([sub_xy, sub_xz, sub_yz], 1)

            U_G = np.concatenate([sub_block1, sub_block2, sub_block3], 1)
            U_I = -np.stack([(rest_layer_points[:, 0] - ref_layer_points[:, 0]), 
                            (rest_layer_points[:, 1] - ref_layer_points[:, 1]),
                            (rest_layer_points[:, 2] - ref_layer_points[:, 2]),
                            (rest_layer_points[:, 0] ** 2 - ref_layer_points[:, 0] ** 2),
                            (rest_layer_points[:, 1] ** 2 - ref_layer_points[:, 1] ** 2),
                            (rest_layer_points[:, 2] ** 2 - ref_layer_points[:, 2] ** 2),
                            (rest_layer_points[:, 0] * rest_layer_points[:, 1] - ref_layer_points[:, 0] * ref_layer_points[:, 1]),
                            (rest_layer_points[:, 0] * rest_layer_points[:, 2] - ref_layer_points[:, 0] * ref_layer_points[:, 2]),
                            (rest_layer_points[:, 1] * rest_layer_points[:, 2] - ref_layer_points[:, 1] * ref_layer_points[:, 2])], 1)

            length_of_CG = C_G.shape[1]
            length_of_CGI = C_GI.shape[1]
            U_G = U_G[:length_of_CG, :3]
            U_I = U_I[:length_of_CGI, :3]


            U = np.concatenate([U_G,U_I],0)

            # build zero matrix
            zero_matrix = np.zeros([U.shape[1],U.shape[1]])

            # concatenate drift matrix and kriging matrix
            K_D = np.concatenate([np.concatenate([K,U],axis = 1),np.concatenate([U.T,zero_matrix],axis = 1)],axis = 0)
            # build right side matrix of cokriging system
            bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
            bk = np.reshape(bk,newshape = [bk.shape[0],1])
            # solve kriging weight
            w = np.linalg.lstsq(K_D,bk)[0]
            # gradient contribution
            sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))

            sigma_0_grad = np.sum(sigma_0_grad,axis=0)
            # surface point contribution
            sigma_0_interf = -w[G_1.shape[0]*3:-3]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
            sigma_0_interf = np.sum(sigma_0_interf,axis = 0)
            # 2nd order drift contribution
            sigma_0_2nd_drift_1 = np.sum(grid* (w[-3:]).T,axis = 1)
            sigma_0_2nd_drift = sigma_0_2nd_drift_1
            interpolate_result = 1*sigma_0_grad+1*sigma_0_interf + 1*sigma_0_2nd_drift 

        intp = interpolate_result  # reshape the result to matrix shape

        if cv == True:
            return intp
        else:
            intp = np.reshape(interpolate_result, [50, 50, 50])
            return intp
    
    def systematic_sampling(self, df, n):
        count = len(df)
        step = max(1, count // n)
        indices = list(range(0, count, step))[:n]
        return df.iloc[indices]
    
    def kmeans_sampling_indices(self,df, n_samples, random_state=None):
        kmeans = KMeans(n_clusters=n_samples, random_state=random_state)
        kmeans.fit(df)
        
        sampled_indices = []
        for i in range(n_samples):
            cluster_indices = (kmeans.labels_ == i)
            cluster_points = df[cluster_indices]
            centroid = kmeans.cluster_centers_[i]
            closest_index = cluster_points.apply(lambda row: ((row - centroid) ** 2).sum(), axis=1).idxmin()
            sampled_indices.append(closest_index)
        
        return sampled_indices
        

    def evaluate_params_kfold(self, a, b, c, eps, range_val, G_position, G_orientation, layer_position, drift_type, n_splits=5):
        try:
            loo = LeaveOneOut()
            kf = KFold(n_splits=n_splits, shuffle=False)
            errors = []
            for train_index, val_index in kf.split(layer_position):
                train_layer = layer_position[train_index]
                val_layer = layer_position[val_index]
                real_data = train_layer[[0]]

                intp = self.RBF_3D_kernel(
                    G_position, G_orientation, train_layer, test_data=val_layer, eps=eps, range_val=range_val,
                    drift=drift_type, cv=True, a=a, b=b, c=c
                )
                actual = self.RBF_3D_kernel(
                    G_position, G_orientation, train_layer, test_data=real_data, eps=eps, range_val=range_val,
                    drift=drift_type, cv=True, a=a, b=b, c=c
                )
                error = np.mean(np.abs(intp - actual)/np.abs(actual))
                errors.append(error)          
            avg_error = np.mean(errors)
            return (avg_error, (a, b, c, eps, range_val))
        except Exception as e:
            print(f"Error with parameters a={a}, b={b}, c={c}: {e}")
            return (np.inf, (a, b, c, eps, range_val))  

    def run_cross_validation(self):
        def squared_euclidean_distance(x_1, x_2):
            sqd = np.sqrt(np.reshape(np.sum(x_1**2, 1), newshape=(x_1.shape[0], 1)) +
                        np.reshape(np.sum(x_2**2, 1), newshape=(1, x_2.shape[0])) -
                        2 * (x_1 @ x_2.T))
            return np.nan_to_num(sqd)
        num_surface_points = int(self.num_surface_points_cv.get())
        dis = squared_euclidean_distance(self.systematic_sampling(self.cv_vertices_df, num_surface_points)[['X','Y','Z']].values, self.systematic_sampling(self.cv_vertices_df, num_surface_points)[['X','Y','Z']].values)
        eps = dis.mean()
        range_val = dis.max() * 2

        try:
            a_start = float(self.param_a_start_cv.get()) if self.param_a_start_cv.get() else 1.0
            a_stop = float(self.param_a_stop_cv.get()) if self.param_a_stop_cv.get() else 2.0
            a_step = float(self.param_a_step_cv.get()) if self.param_a_step_cv.get() else 1.0

            b_start = float(self.param_b_start_cv.get()) if self.param_b_start_cv.get() else 1.0
            b_stop = float(self.param_b_stop_cv.get()) if self.param_b_stop_cv.get() else 2.0
            b_step = float(self.param_b_step_cv.get()) if self.param_b_step_cv.get() else 1.0

            c_start = float(self.param_c_start_cv.get()) if self.param_c_start_cv.get() else 1.0
            c_stop = float(self.param_c_stop_cv.get()) if self.param_c_stop_cv.get() else 2.0
            c_step = float(self.param_c_step_cv.get()) if self.param_c_step_cv.get() else 1.0

            eps_start = float(self.epsilon_start_cv.get()) if self.epsilon_start_cv.get() else eps
            eps_stop = float(self.epsilon_stop_cv.get()) if self.epsilon_stop_cv.get() else eps*2
            eps_step = float(self.epsilon_step_cv.get()) if self.epsilon_step_cv.get() else eps

            range_start = float(self.range_start_cv.get()) if self.range_start_cv.get() else range_val
            range_stop = float(self.range_stop_cv.get()) if self.range_stop_cv.get() else range_val*2
            range_step = float(self.range_step_cv.get()) if self.range_step_cv.get() else range_val


            a_values = np.arange(a_start, a_stop, a_step)
            b_values = np.arange(b_start, b_stop, b_step)
            c_values = np.arange(c_start, c_stop, c_step)
            eps_values = np.arange(eps_start, eps_stop, eps_step)
            range_values = np.arange(range_start, range_stop, range_step)

            param_combinations = list(itertools.product(a_values, b_values, c_values, eps_values, range_values))
            total_combinations = len(param_combinations)

            best_error = np.inf
            best_params = None

            self.progress['maximum'] = total_combinations
            self.progress['value'] = 0
            start_time = time.time()

            num_surface_points = int(self.num_surface_points_cv.get())
            num_orientation_points = int(self.num_orientation_points_cv.get())
            random_state = 42


            surface_sampled_indices = self.kmeans_sampling_indices(self.cv_vertices_df, num_surface_points, random_state)
            if self.Sampling_method_cv.get() == 'Statistic(K-means)':
                layer_position = self.cv_vertices_df.iloc[surface_sampled_indices][['X', 'Y', 'Z']].values
                G_position = layer_position[:num_orientation_points]
                G_orientation = self.cv_normals_df.iloc[surface_sampled_indices][['x', 'y', 'z']].values[:num_orientation_points]
            elif self.Sampling_method_cv.get() == 'Random':
                layer_position = self.cv_vertices_df.sample(n=num_surface_points, random_state=random_state)[['X', 'Y', 'Z']].values
                G_position = self.cv_vertices_df.sample(n=num_orientation_points, random_state=random_state)[['X', 'Y', 'Z']].values
                G_orientation = self.cv_normals_df.sample(n=num_orientation_points, random_state=random_state)[['x', 'y', 'z']].values
            elif self.Sampling_method_cv.get() == 'Systematic':
                layer_position = self.systematic_sampling(self.cv_vertices_df, num_surface_points)[['X', 'Y', 'Z']].values
                G_position = self.systematic_sampling(self.cv_vertices_df, num_orientation_points)[['X', 'Y', 'Z']].values
                G_orientation = self.systematic_sampling(self.cv_normals_df, num_orientation_points)[['x', 'y', 'z']].values

            drift_type_str = self.drift_type_cv.get()
            drift_type_map = {
                'None': 0,
                'Second Order Polynomial': 1,
                'Ellipsoid': 2,
                'Dome Shaped': 3,
                'First Order Polynomial': 4
            }
            drift_type = drift_type_map.get(drift_type_str, 0)

            for idx, (a, b, c, eps, range_val) in enumerate(param_combinations):
                error, params = self.evaluate_params_kfold(a, b, c, eps, range_val, G_position, G_orientation, layer_position, drift_type, n_splits=5)
                if error < best_error:
                    best_error = error
                    best_params = params

                self.progress['value'] = idx + 1
                elapsed_time = time.time() - start_time
                estimated_total_time = (elapsed_time / (idx+1)) * total_combinations
                remaining_time = estimated_total_time - elapsed_time
                self.elapsed_time_label.config(text=f"time used:{int(elapsed_time)}s")
                self.remaining_time_label.config(text=f"estimated remaining time:{int(remaining_time)}s")
                self.root.update_idletasks()
            
            if best_params is not None:
                self.result_label.config(
                    text = f'Best parameters: eps={best_params[3]:.2f}, range={best_params[4]:.2f}, a={best_params[0]:.2f}, b={best_params[1]:.2f}, c={best_params[2]:.2f}\nLeast error: {best_error:.2f} %'
                )
            else:
                self.result_label.config(text="Cant find best parameters.")
        except Exception as e:
            error_message = traceback.format_exc()
            messagebox.showerror("Error", f"Error: {e}\n\n{error_message}")

    def compute_rbf_interpolation(self):

        def squared_euclidean_distance(x_1, x_2):
                    sqd = np.sqrt(np.reshape(np.sum(x_1**2, 1), newshape=(x_1.shape[0], 1)) +
                                np.reshape(np.sum(x_2**2, 1), newshape=(1, x_2.shape[0])) -
                                2 * (x_1 @ x_2.T))
                    return np.nan_to_num(sqd)
        
        if self.vertices_df is None or self.normals_o_df is None:
            messagebox.showerror("Error", "Please load both surface points and orientations.")
            return
        
        if self.normals_df is None:
            self.normals_df = self.vertices_df
            return

        try:

            seed = 55
            num_surface_points = int(self.num_surface_points.get())
            num_orientation_points = int(self.num_orientation_points.get())
            kernel_function = self.kernel_function.get()
            model_color = self.model_color.get()
            drift_type_str = self.drift_type.get()


            # Default values for eps, range, a, b, c
            dis = squared_euclidean_distance(self.systematic_sampling(self.vertices_df, num_surface_points)[['X','Y','Z']].values, self.systematic_sampling(self.vertices_df, num_surface_points)[['X','Y','Z']].values)
            default_eps = dis.mean()
            default_range = dis.max() * 2
            default_a = 1
            default_b = 1
            default_c = 1

            # Use default values if entries are empty
            eps = float(self.epsilon_entry.get()) if self.epsilon_entry.get() else default_eps
            range_val = float(self.range_entry.get()) if self.range_entry.get() else default_range
            a = float(self.param_a_entry.get()) if self.param_a_entry.get() else default_a
            b = float(self.param_b_entry.get()) if self.param_b_entry.get() else default_b
            c = float(self.param_c_entry.get()) if self.param_c_entry.get() else default_c

            

            drift_type_map = {
                'None': 0,
                'Second Order Polynomial': 1,
                'Ellipsoid': 2,
                'Dome Shaped': 3,
                'First Order Polynomial': 4,
                '2D Custom' : 5,
                '3D Custom' : 6
            }
            drift_type = drift_type_map[drift_type_str]

            def kernel(radius, function, eps=1, range_val=1, a=1, b=1, n=1):
                if function == 'cubic':
                    return radius**3
                elif function == 'gaussian':
                    return np.exp(-1 * (radius**2)/ (2* eps**2))
                elif function == 'quintic':
                    return radius**5
                elif function == 'cubic_covariance':
                    return (1 - 7 * (radius / range_val) ** 2 +
                            35 / 4 * (radius / range_val) ** 3 -
                            7 / 2 * (radius / range_val) ** 5 +
                            3 / 4 * (radius / range_val) ** 7)
                elif function == 'multiquadric':
                    return np.sqrt(radius**2+eps**2)
                elif function == 'linear':
                    return -radius
                elif function == 'thin_plate':
                    return radius**2 * np.log(radius)
                elif function == 'inverse_quadratic':
                    return 1 / (eps**2 + radius**2)
                elif function == 'inverse_multiquadric':
                    return 1 / np.sqrt(radius**2+eps**2)
                elif function == 'gaussian_covariance':
                    return np.exp(-(radius**2 / range_val**2))
                elif function == 'ga_cub_cov':
                    return a * (1 - np.exp(-(radius / range_val)**2)) + b * (1 - 7 * (radius / range_val) ** 2 +
                        35 / 4 * (radius / range_val) ** 3 - 7 / 2 * (radius / range_val) ** 5 +
                        3 / 4 * (radius / range_val) ** 7)
                elif function == 'ga_cub':
                    return np.exp(-1 * radius**2 / (2 * eps**2)) + n * (radius**3)

            # Perform RBF Interpolation
            def RBF_3D_kernel(G_position, G_orientation, layer_position, test_data, drift=0, cv=True):
                df = egrad(kernel)
                ddf = egrad(df)
                func = kernel_function
                G_1 = G_position
                G_1_o = G_orientation
                layer1 = layer_position

                def squared_euclidean_distance(x_1, x_2):
                    sqd = np.sqrt(np.reshape(np.sum(x_1**2, 1), newshape=(x_1.shape[0], 1)) +
                                np.reshape(np.sum(x_2**2, 1), newshape=(1, x_2.shape[0])) -
                                2 * (x_1 @ x_2.T))
                    return np.nan_to_num(sqd)

                def cartesian_dist(x_1, x_2):
                    return np.concatenate([
                        np.tile(x_1[:, 0] - np.reshape(x_2[:, 0], [x_2[:, 0].shape[0], 1]), [1, 3]),
                        np.tile(x_1[:, 1] - np.reshape(x_2[:, 1], [x_2[:, 1].shape[0], 1]), [1, 3]),
                        np.tile(x_1[:, 2] - np.reshape(x_2[:, 2], [x_2[:, 2].shape[0], 1]), [1, 3])], axis=0)

                def cov_gradients(dist_tiled):
                    a = (h_u * h_v)
                    b = dist_tiled**2

                    t1 = np.divide(a, b, out=np.zeros_like(a), casting='unsafe', where=b!=0)
                    t2 = np.where(dist_tiled < range_val, ddf(dist_tiled, function=func) - np.nan_to_num(df(dist_tiled, function=func)/dist_tiled), 0)
                    t3 = perpendicularity_matrix * np.where(dist_tiled < range_val, (np.nan_to_num(df(dist_tiled, function=func)/dist_tiled)), 0)
                    t4 = 1/3 * np.eye(dist_tiled.shape[0])

                    C_G = t1 * t2 - t3 + t4
                    return C_G

                def set_rest_ref_matrix(number_of_points_per_surface):
                    ref_layer_points = np.repeat(np.stack([layer1[-1]], axis=0), repeats=number_of_points_per_surface-1, axis=0)
                    rest_layer_points = np.concatenate([layer1[0:-1]], axis=0)
                    return ref_layer_points, rest_layer_points

                def cov_interface(ref_layer_points, rest_layer_points):
                    sed_rest_rest = squared_euclidean_distance(rest_layer_points, rest_layer_points)
                    sed_ref_rest = squared_euclidean_distance(ref_layer_points, rest_layer_points)
                    sed_rest_ref = squared_euclidean_distance(rest_layer_points, ref_layer_points)
                    sed_ref_ref = squared_euclidean_distance(ref_layer_points, ref_layer_points)

                    C_I = kernel(sed_rest_rest, function=func) - kernel(sed_ref_rest, function=func) - kernel(sed_rest_ref, function=func) + kernel(sed_ref_ref, function=func)
                    return C_I

                def cartesian_dist_no_tile(x_1, x_2):
                    return np.concatenate([
                        np.transpose((x_1[:, 0] - np.reshape(x_2[:, 0], [x_2.shape[0], 1]))),
                        np.transpose((x_1[:, 1] - np.reshape(x_2[:, 1], [x_2.shape[0], 1]))),
                        np.transpose((x_1[:, 2] - np.reshape(x_2[:, 2], [x_2.shape[0], 1])))
                    ], axis=0)

                def cov_interface_gradients(hu_rest, hu_ref):
                    C_GI = hu_rest * np.where(sed_dips_rest < range_val, (np.nan_to_num(df(sed_dips_rest, function=func) / sed_dips_rest)), 0) - \
                        hu_ref * np.where(sed_dips_ref < range_val, (np.nan_to_num(df(sed_dips_ref, function=func) / sed_dips_ref)), 0)
                    return C_GI

                def perpendicularity(G_1):
                    a = np.concatenate([np.ones([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]])], axis=1)
                    b = np.concatenate([np.zeros([G_1.shape[0], G_1.shape[0]]), np.ones([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]])], axis=1)
                    c = np.concatenate([np.zeros([G_1.shape[0], G_1.shape[0]]), np.zeros([G_1.shape[0], G_1.shape[0]]), np.ones([G_1.shape[0], G_1.shape[0]])], axis=1)
                    return np.concatenate([a, b, c], axis=0)
                
                def plot_3D(grid,value,surfaces_nr=10):
                    pv.set_jupyter_backend('static')
                    grid = pv.StructuredGrid(XX,YY,ZZ)
                    p = pv.Plotter(notebook=True,window_size=[1500,1500])
                    p.set_background('white')

                    p.add_points(layer1[:,[0,1,2]], render_points_as_spheres=True,point_size=20.0,color='blue')

                    grid.point_data['scalar'] = value.ravel(order='F')

                    index = np.where(layer_position[:,0] + layer_position[:,1] == (layer_position[:,0] + layer_position[:,1]).min())
                    distances = np.sqrt((XX - layer1[index,0])**2 + (YY - layer1[index,1])**2 + (ZZ - layer1[index,2])**2)
                    closest_point_indices = np.unravel_index(np.argmin(distances), distances.shape)
                    lvl_t = np.array([intp[closest_point_indices]])
                    print('surface scalar value:',lvl_t)
                    contours_2 = grid.contour(lvl_t)
                    p.add_mesh(contours_2, show_scalar_bar=0, label='surface',style='surface',color=model_color,opacity=0.8)

                    p.add_bounding_box()

                    p.show_bounds(
                        grid='front',  
                        location='outer',  
                        xlabel='X Axis',
                        ylabel='Y Axis',
                        zlabel='Z Axis',
                        color='black',  
                        font_size=30  
                    )
                    p.set_scale(1,1,1.5)
                    p.add_arrows((G_1[:,[0,1,2]]),direction=(G_1_o[:,[0,1,2]]),color='black',mag=50)
                    p.add_axes()
                    p.screenshot()
                    p.show()
                    return contours_2
                
                def plot_3D_drift(grid,value,surfaces_nr=10):
                    pv.set_jupyter_backend('static')

                    grid = pv.StructuredGrid(XX,YY,ZZ)

                    p = pv.Plotter(notebook=True,window_size=[400,400])
                    p.set_background('white')

                    p.add_points(layer1[:,[0,1,2]], render_points_as_spheres=True,point_size=14.0,color='blue')

                    grid.point_data['scalar'] = value.ravel(order='F')
                    contours_1 = grid.contour(np.linspace(value.min(),value.max(),surfaces_nr))
                    p.add_mesh(contours_1, show_scalar_bar=0, label='scalar_field',style='surface',opacity=0.8)
                    p.add_arrows((G_1[:,[0,1,2]]),direction=(G_1_o[:,[0,1,2]]),color='black',mag=50)
                    p.add_axes()
                    p.show()
                    return contours_1
                
                

                
                G_1_tiled = np.tile(G_1, [3, 1])

                h_u = cartesian_dist(G_1, G_1)
                h_v = h_u.T

                perpendicularity_matrix = perpendicularity(G_1)

                dist_tiled = squared_euclidean_distance(G_1_tiled, G_1_tiled)

                dist_tiled = dist_tiled + np.eye(dist_tiled.shape[0])

                C_G = cov_gradients(dist_tiled)

                number_of_points_per_surface = np.array([layer1.shape[0]])

                ref_layer_points, rest_layer_points = set_rest_ref_matrix(number_of_points_per_surface)

                C_I = cov_interface(ref_layer_points, rest_layer_points)

                sed_dips_rest = squared_euclidean_distance(G_1_tiled, rest_layer_points)
                sed_dips_ref = squared_euclidean_distance(G_1_tiled, ref_layer_points)

                hu_rest = cartesian_dist_no_tile(G_1, rest_layer_points)
                hu_ref = cartesian_dist_no_tile(G_1, ref_layer_points)

                C_GI = cov_interface_gradients(hu_rest, hu_ref)
                C_IG = C_GI.T

                x0 = (layer1[:, 0].min() + layer1[:, 0].max()) / 2
                y0 = (layer1[:, 1].min() + layer1[:, 1].max()) / 2
                z0 = (layer1[:, 2].min() + layer1[:, 2].max()) / 2
                frame_x = (layer1[:, 0].max() - layer1[:, 0].min()) / 10
                frame_y = (layer1[:, 1].max() - layer1[:, 1].min()) / 10
                frame_z = (layer1[:, 2].max() - layer1[:, 2].min()) / 2
                xx = np.linspace(layer1[:, 0].min() - frame_x, layer1[:, 0].max() + frame_x, 50)
                yy = np.linspace(layer1[:, 1].min() - frame_y, layer1[:, 1].max() + frame_y, 50)
                zz = np.linspace(layer1[:, 2].min() - frame_z, layer1[:, 2].max() + frame_z, 50)
                XX, YY, ZZ = np.meshgrid(xx, yy, zz)
                X = np.reshape(XX, [-1]).T
                Y = np.reshape(YY, [-1]).T
                Z = np.reshape(ZZ, [-1]).T
                if cv == True:
                    grid = test_data
                else:
                    grid = np.stack([X, Y, Z], axis=1)

                hu_Simpoints = cartesian_dist_no_tile(G_1, grid)
                sed_dips_SimPoint = squared_euclidean_distance(G_1_tiled, grid)
                sed_rest_SimPoint = squared_euclidean_distance(rest_layer_points, grid)
                sed_ref_SimPoint = squared_euclidean_distance(ref_layer_points, grid)

                if drift == 1:  # 'Second Order Polynomial'
                    K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
                    n = G_1.shape[0]
                    sub_x = np.tile(np.array([[1,0,0]]),[n,1])
                    sub_y = np.tile(np.array([[0,1,0]]),[n,1])
                    sub_z = np.tile(np.array([[0,0,1]]),[n,1])
                    sub_block1 = np.concatenate([sub_x,sub_y,sub_z],0)

                    sub_x_2 = np.zeros((n, 3))
                    sub_y_2 = np.zeros((n, 3))
                    sub_z_2 = np.zeros((n, 3))
                    sub_x_2[:, 0] = 2 * G_1[:, 0]
                    sub_y_2[:, 1] = 2 * G_1[:, 1]
                    sub_z_2[:, 2] = 2 * G_1[:, 2]

                    sub_block2 = np.concatenate([sub_x_2, sub_y_2, sub_z_2], 0)

                    sub_xy = np.reshape(np.concatenate([G_1[:, 1],G_1[:, 0]], 0), [2 * n, 1])
                    sub_xy = np.pad(sub_xy, [[0, n], [0, 0]])

                    sub_xz = np.concatenate([np.pad(np.reshape(G_1[:, 2], [n, 1]), [[0, n], [0, 0]]), np.reshape(G_1[:, 0], [n, 1])], 0)

                    sub_yz = np.reshape(np.concatenate([G_1[:, 2], G_1[:, 1]], 0), [2 * n, 1])
                    sub_yz = np.pad(sub_yz, [[n, 0], [0, 0]])

                    sub_block3 = np.concatenate([sub_xy, sub_xz, sub_yz], 1)

                    U_G = np.concatenate([sub_block1, sub_block2, sub_block3], 1)
                    U_I = -np.stack([(rest_layer_points[:, 0] - ref_layer_points[:, 0]), 
                                    (rest_layer_points[:, 1] - ref_layer_points[:, 1]),
                                    (rest_layer_points[:, 2] - ref_layer_points[:, 2]),
                                    (rest_layer_points[:, 0] ** 2 - ref_layer_points[:, 0] ** 2),
                                    (rest_layer_points[:, 1] ** 2 - ref_layer_points[:, 1] ** 2),
                                    (rest_layer_points[:, 2] ** 2 - ref_layer_points[:, 2] ** 2),
                                    (rest_layer_points[:, 0] * rest_layer_points[:, 1] - ref_layer_points[:, 0] * ref_layer_points[:, 1]),
                                    (rest_layer_points[:, 0] * rest_layer_points[:, 2] - ref_layer_points[:, 0] * ref_layer_points[:, 2]),
                                    (rest_layer_points[:, 1] * rest_layer_points[:, 2] - ref_layer_points[:, 1] * ref_layer_points[:, 2])], 1)

                    length_of_CG = C_G.shape[1]
                    length_of_CGI = C_GI.shape[1]
                    U_G = U_G[:length_of_CG, :9]
                    U_I = U_I[:length_of_CGI, :9]


                    U = np.concatenate([U_G,U_I],0)

                    # build zero matrix
                    zero_matrix = np.zeros([U.shape[1],U.shape[1]])

                    # concatenate drift matrix and kriging matrix
                    K_D = np.concatenate([np.concatenate([K,U],axis = 1),np.concatenate([U.T,zero_matrix],axis = 1)],axis = 0)
                    # build right side matrix of cokriging system
                    bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
                    bk = np.reshape(bk,newshape = [bk.shape[0],1])
                    # solve kriging weight
                    w = np.linalg.lstsq(K_D,bk)[0]
                    # gradient contribution
                    sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))

                    sigma_0_grad = np.sum(sigma_0_grad,axis=0)
                    # surface point contribution
                    sigma_0_interf = -w[G_1.shape[0]*3:-9]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
                    sigma_0_interf = np.sum(sigma_0_interf,axis = 0)
                    # 2nd order drift contribution
                    sigma_0_2nd_drift_1 = grid[:,0] * w[-9] + grid[:,1] * w[-8] + grid[:,2] * w[-7] + \
                        grid[:,0] ** 2 * w[-6] + grid[:,1] ** 2 * w[-5] + grid[:,2] ** 2 * w[-4] + \
                        grid[:,0] * grid[:,1] * w[-3] + grid[:,0] * grid[:,2] * w[-2] + grid[:,1] * grid[:,2] * w[-1]
                    sigma_0_2nd_drift = sigma_0_2nd_drift_1
                    np.save("Drift contribution",sigma_0_2nd_drift )
                    np.save("Gradient contribution",sigma_0_grad)
                    np.save("Interface contribution",sigma_0_interf)
                    interpolate_result = 1*sigma_0_grad+1*sigma_0_interf + 1*sigma_0_2nd_drift 
                    drift_result = 0*sigma_0_grad+0*sigma_0_interf + 1*sigma_0_2nd_drift

                elif drift == 0:  # 'None'
                    K_D = np.nan_to_num(np.concatenate([np.concatenate([C_G, C_GI], axis=1), np.concatenate([C_IG, C_I], axis=1)], axis=0))

                    bk = np.concatenate([G_1_o[:, 0], G_1_o[:, 1], G_1_o[:, 2], np.zeros(K_D.shape[0] - G_1.shape[0] * 3)], axis=0)
                    bk = np.reshape(bk, newshape=[bk.shape[0], 1])

                    w = np.linalg.lstsq(K_D, bk, rcond=None)[0]
                    sigma_0_grad = w[:G_1.shape[0] * 3] * (-hu_Simpoints * (sed_dips_SimPoint < range_val) * (np.nan_to_num(df(sed_dips_SimPoint, function=func) / sed_dips_SimPoint)))

                    sigma_0_grad = np.sum(sigma_0_grad, axis=0)
                    sigma_0_interf = -w[G_1.shape[0] * 3:] * (((sed_rest_SimPoint < range_val) * (kernel(sed_rest_SimPoint, function=func)) - (sed_ref_SimPoint < range_val) * (kernel(sed_ref_SimPoint, function=func))))
                    sigma_0_interf = np.sum(sigma_0_interf, axis=0)

                    interpolate_result = 1 * sigma_0_grad + 1 * sigma_0_interf
                    drift_result = 0 * sigma_0_grad + 0 * sigma_0_interf

                elif drift == 2:  # 'Spherical'
                    x0 = (self.vertices_df[['X']].values.max()-self.vertices_df[['X']].values.min())/2
                    y0 = (self.vertices_df[['Y']].values.max()-self.vertices_df[['Y']].values.min())/2
                    z0 = (self.vertices_df[['Z']].values.max()-self.vertices_df[['Z']].values.min())/2
                    r = 1
                    K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
                    D_Z = ((2*(G_1[:,2]-z0)/c)**2).reshape(G_1.shape[0],1)
                    D_X = ((2*(G_1[:,1]-x0)/a)**2).reshape(G_1.shape[0],1)
                    D_Y = ((2*(G_1[:,0]-y0)/b)**2).reshape(G_1.shape[0],1)

                    D_I = ((((ref_layer_points[:,0]-x0)/a)**2+((ref_layer_points[:,1]-y0)/b)**2+((ref_layer_points[:,2]-z0)/c)**2-r**2) \
                        - (((rest_layer_points[:,0]-x0)/a)**2+((rest_layer_points[:,1]-y0)/b)**2+((rest_layer_points[:,2]-z0)/c)**2-r**2)).reshape(-1,1)
                
                    # build external drift matrix
                    D_E = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
                    D_E_T = D_E.T

                    D = D_E
                    D_T = D.T 

                    # build zero matrix
                    zero_matrix = np.zeros([D.shape[1],D.shape[1]])
                    # concatenate drift matrix and kriging matrix
                    K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)

                    # build right side matrix of cokriging system
                    bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
                    bk = np.reshape(bk,newshape = [bk.shape[0],1])

                    # solve kriging weight
                    w = np.linalg.lstsq(K_D,bk)[0]
                    # gradient contribution
                    sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))

                    sigma_0_grad = np.sum(sigma_0_grad,axis=0)
                    # surface point contribution
                    sigma_0_interf = -w[G_1.shape[0]*3:-1]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
                    sigma_0_interf = np.sum(sigma_0_interf,axis = 0)


                    external_drift = (((grid[:,0]-x0)/a)**2+((grid[:,1]-y0)/b)**2+((grid[:,2]-z0)/c)**2+r**2) * (w[-1]).T
                    interpolate_result = 1*sigma_0_grad+1*sigma_0_interf   + 1* external_drift
                    drift_result = 0*sigma_0_grad+0*sigma_0_interf + 1*external_drift


                elif drift == 3:  # 'Dome Shaped'
                    K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
                    # z - (c - (x**2 + y**2) / a**2) * np.exp(-(x**2 + y**2) / b**2)

                    def F_x(x, y, z, a, b, c):
                        return z - (c - ((x - x0)**2 + (y - y0)**2) / a**2) * np.exp(-((x - x0)**2 + (y - y0)**2) / b**2)
                    
                    def partial_F_x(x, y, a, b, c):
                        term1 = (2 * (x - x0) * (c - ((x - x0)**2 + (y - y0)**2) / a**2)) / b**2
                        term2 = (2 * (x - x0)) / a**2
                        return (term1 + term2) * np.exp(-((x - x0)**2 + (y - y0)**2) / b**2)

                    def partial_F_y(y, x, a, b, c):
                        term1 = (2 * (y - y0) * (c - ((x - x0)**2 + (y - y0)**2) / a**2)) / b**2
                        term2 = (2 * (y - y0)) / a**2
                        return (term1 + term2) * np.exp(-((x - x0)**2 + (y - y0)**2) / b**2)

                    def partial_F_z():
                        return np.ones_like(x)
                    
                    x = G_1[:, 0]
                    y = G_1[:, 1]
                    z = G_1[:, 2]

                    D_Z = partial_F_z().reshape(G_1.shape[0],1)
                    D_X = partial_F_x(x, y, a, b, c).reshape(G_1.shape[0],1)
                    D_Y = partial_F_y(y, x, a, b, c).reshape(G_1.shape[0],1)

                    D_I = (F_x(ref_layer_points[:,0], ref_layer_points[:,1], ref_layer_points[:,2], a, b, c) \
                        - F_x(rest_layer_points[:,0], rest_layer_points[:,1], rest_layer_points[:,2], a, b, c)).reshape(-1,1)
                    # build external drift matrix
                    D = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)

                    D_T = D.T 

                    # build zero matrix
                    zero_matrix = np.zeros([D.shape[1],D.shape[1]])
                    # concatenate drift matrix and kriging matrix
                    K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)

                    # build right side matrix of cokriging system
                    bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
                    bk = np.reshape(bk,newshape = [bk.shape[0],1])

                    # solve kriging weight
                    w = np.linalg.lstsq(K_D,bk)[0]
                    # gradient contribution
                    sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))

                    sigma_0_grad = np.sum(sigma_0_grad,axis=0)
                    # surface point contribution
                    sigma_0_interf = -w[G_1.shape[0]*3:-1]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
                    sigma_0_interf = np.sum(sigma_0_interf,axis = 0)


                    external_drift = F_x(grid[:,0],grid[:,1],grid[:,2],a,b,c) * (w[-1]).T
                    interpolate_result = 1*sigma_0_grad+1*sigma_0_interf   + 1* external_drift
                    drift_result = 0*sigma_0_grad+0*sigma_0_interf + 1*external_drift

                elif drift == 4:  # 'First Order Polynomial'
                    K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
                    n = G_1.shape[0]
                    sub_x = np.tile(np.array([[1,0,0]]),[n,1])
                    sub_y = np.tile(np.array([[0,1,0]]),[n,1])
                    sub_z = np.tile(np.array([[0,0,1]]),[n,1])
                    sub_block1 = np.concatenate([sub_x,sub_y,sub_z],0)

                    sub_x_2 = np.zeros((n, 3))
                    sub_y_2 = np.zeros((n, 3))
                    sub_z_2 = np.zeros((n, 3))
                    sub_x_2[:, 0] = 2 * G_1[:, 0]
                    sub_y_2[:, 1] = 2 * G_1[:, 1]
                    sub_z_2[:, 2] = 2 * G_1[:, 2]

                    sub_block2 = np.concatenate([sub_x_2, sub_y_2, sub_z_2], 0)

                    sub_xy = np.reshape(np.concatenate([G_1[:, 1],G_1[:, 0]], 0), [2 * n, 1])
                    sub_xy = np.pad(sub_xy, [[0, n], [0, 0]])

                    sub_xz = np.concatenate([np.pad(np.reshape(G_1[:, 2], [n, 1]), [[0, n], [0, 0]]), np.reshape(G_1[:, 0], [n, 1])], 0)

                    sub_yz = np.reshape(np.concatenate([G_1[:, 2], G_1[:, 1]], 0), [2 * n, 1])
                    sub_yz = np.pad(sub_yz, [[n, 0], [0, 0]])

                    sub_block3 = np.concatenate([sub_xy, sub_xz, sub_yz], 1)

                    U_G = np.concatenate([sub_block1, sub_block2, sub_block3], 1)
                    U_I = -np.stack([(rest_layer_points[:, 0] - ref_layer_points[:, 0]), 
                                    (rest_layer_points[:, 1] - ref_layer_points[:, 1]),
                                    (rest_layer_points[:, 2] - ref_layer_points[:, 2]),
                                    (rest_layer_points[:, 0] ** 2 - ref_layer_points[:, 0] ** 2),
                                    (rest_layer_points[:, 1] ** 2 - ref_layer_points[:, 1] ** 2),
                                    (rest_layer_points[:, 2] ** 2 - ref_layer_points[:, 2] ** 2),
                                    (rest_layer_points[:, 0] * rest_layer_points[:, 1] - ref_layer_points[:, 0] * ref_layer_points[:, 1]),
                                    (rest_layer_points[:, 0] * rest_layer_points[:, 2] - ref_layer_points[:, 0] * ref_layer_points[:, 2]),
                                    (rest_layer_points[:, 1] * rest_layer_points[:, 2] - ref_layer_points[:, 1] * ref_layer_points[:, 2])], 1)

                    length_of_CG = C_G.shape[1]
                    length_of_CGI = C_GI.shape[1]
                    U_G = U_G[:length_of_CG, :3]
                    U_I = U_I[:length_of_CGI, :3]


                    U = np.concatenate([U_G,U_I],0)

                    # build zero matrix
                    zero_matrix = np.zeros([U.shape[1],U.shape[1]])

                    # concatenate drift matrix and kriging matrix
                    K_D = np.concatenate([np.concatenate([K,U],axis = 1),np.concatenate([U.T,zero_matrix],axis = 1)],axis = 0)
                    # build right side matrix of cokriging system
                    bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
                    bk = np.reshape(bk,newshape = [bk.shape[0],1])
                    # solve kriging weight
                    w = np.linalg.lstsq(K_D,bk)[0]
                    # print(w.shape)
                    # gradient contribution
                    sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))
                    sigma_0_grad = np.sum(sigma_0_grad,axis=0)
                    # surface point contribution
                    sigma_0_interf = -w[G_1.shape[0]*3:-3]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
                    sigma_0_interf = np.sum(sigma_0_interf,axis = 0)
                    # 2nd order drift contribution
                    sigma_0_2nd_drift_1 = np.sum(grid* (w[-3:]).T,axis = 1)
                    sigma_0_2nd_drift = sigma_0_2nd_drift_1
                    np.save("Drift contribution",sigma_0_2nd_drift )
                    np.save("Gradient contribution",sigma_0_grad)
                    np.save("Interface contribution",sigma_0_interf)
                    interpolate_result = 1*sigma_0_grad+1*sigma_0_interf + 1*sigma_0_2nd_drift 
                    drift_result = 0*sigma_0_grad+0*sigma_0_interf + 1*sigma_0_2nd_drift

                elif drift == 5:  # '2D custom'
                    gravity, aa, bb, cc, dd, ee, ff = self.gravity_fwd_2d()                  
                    def F_x(x, y, aa, bb, cc, dd, ee, ff):
                        return aa * x**2 + bb * y**2 + cc * x * y + dd * x + ee * y + ff
                    
                    def partial_F_x(x, y, aa, bb, cc, dd, ee, ff):
                        return 2 * aa * x + cc * y + dd
                    
                    def partial_F_y(x, y, aa, bb, cc, dd, ee, ff):
                        return 2 * bb * y + cc * x + ee

                    K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)

                    x = G_1[:, 0]
                    y = G_1[:, 1]
                    z = G_1[:, 2]

                    D_X = partial_F_x(x, y, aa, bb, cc, dd, ee, ff).reshape(G_1.shape[0],1)
                    D_Y = partial_F_y(x, y, aa, bb, cc, dd, ee, ff).reshape(G_1.shape[0],1)
                    D_Z = np.ones_like(x).reshape(G_1.shape[0],1) * 0
                    D_I = (F_x(ref_layer_points[:,0], ref_layer_points[:,1], aa, bb, cc, dd, ee, ff) \
                        - F_x(rest_layer_points[:,0], rest_layer_points[:,1], aa, bb, cc, dd, ee, ff)).reshape(-1,1)
                
                    # build external drift matrix
                    D_E = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
                    D = D_E
                    D_T = D.T 

                    # build zero matrix
                    zero_matrix = np.zeros([D.shape[1],D.shape[1]])
                    # concatenate drift matrix and kriging matrix
                    K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)

                    # build right side matrix of cokriging system
                    bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
                    bk = np.reshape(bk,newshape = [bk.shape[0],1])

                    # solve kriging weight
                    w = np.linalg.lstsq(K_D,bk)[0]
                    # gradient contribution
                    sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))

                    sigma_0_grad = np.sum(sigma_0_grad,axis=0)
                    # surface point contribution
                    sigma_0_interf = -w[G_1.shape[0]*3:-1]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
                    sigma_0_interf = np.sum(sigma_0_interf,axis = 0)

                    
                    external_drift = F_x(grid[:,0],grid[:,1],aa,bb,cc,dd,ee,ff) * (w[-1]).T
                    interpolate_result = 1*sigma_0_grad+1*sigma_0_interf   + 1* external_drift
                    drift_result = 0*sigma_0_grad+0*sigma_0_interf + 1*external_drift

                elif drift == 6:  # '3D custom'

                    def numerical_gradient(F, x, eps=1e-5):
                        # F is a scalar function from R^n -> R
                        # x is a point in R^n
                        grad = np.zeros_like(x, dtype=float)
                        for i in range(len(x)):
                            x_plus = x.copy();   x_plus[i] += eps
                            x_minus = x.copy();  x_minus[i] -= eps
                            grad[i] = (F(x_plus) - F(x_minus)) / (2*eps)
                        return grad

                    
                    K = np.concatenate([np.concatenate([C_G,C_GI],axis = 1),np.concatenate([C_IG,C_I],axis = 1)],axis = 0)
                    scalar_field = self.drift_scalar 
                    scalar_field *= float(self.scaling_factor.get())
                    scalar_field =  scalar_field.transpose(2, 1, 0)

                    interpolator = RegularGridInterpolator((xx,yy,zz), scalar_field,method='linear')
                    def F(point):
                        return interpolator(point.reshape(1, -1))[0]   
                    def F_x(points):
                        return interpolator(points) 

                    gradient_F = egrad(F)  

                    points = G_1  

                    gradients = np.array([numerical_gradient(F, p) for p in points])

                    D_X = gradients[:, 0].reshape(-1, 1)
                    D_Y = gradients[:, 1].reshape(-1, 1)
                    D_Z = gradients[:, 2].reshape(-1, 1)
                    D_I = (F_x(np.array(ref_layer_points) ) - F_x(np.array(rest_layer_points))).reshape(-1, 1)
                    # build external drift matrix
                    D = np.concatenate([D_X , D_Y , D_Z , D_I],axis=0)
                    D_T = D.T 
                    # build zero matrix
                    zero_matrix = np.zeros([D.shape[1],D.shape[1]])
                    # concatenate drift matrix and kriging matrix
                    K_D = np.concatenate([np.concatenate([K,D],axis = 1),np.concatenate([D_T,zero_matrix],axis = 1)],axis = 0)

                    # build right side matrix of cokriging system
                    bk = np.concatenate([G_1_o[:,0],G_1_o[:,1],G_1_o[:,2],np.zeros(K_D.shape[0]-G_1.shape[0]*3)],axis = 0)
                    bk = np.reshape(bk,newshape = [bk.shape[0],1])

                    # solve kriging weight
                    w = np.linalg.lstsq(K_D,bk)[0]
                    # gradient contribution
                    sigma_0_grad = w[:G_1.shape[0]*3] * (-hu_Simpoints*(sed_dips_SimPoint < range_val)*(np.nan_to_num(df(sed_dips_SimPoint,function = func)/sed_dips_SimPoint)))

                    sigma_0_grad = np.sum(sigma_0_grad,axis=0)
                    # surface point contribution
                    sigma_0_interf = -w[G_1.shape[0]*3:-1]*(((sed_rest_SimPoint < range_val)*(kernel(sed_rest_SimPoint,function = func)) - (sed_ref_SimPoint < range_val)*(kernel(sed_ref_SimPoint,function = func))))
                    sigma_0_interf = np.sum(sigma_0_interf,axis = 0)


                    external_drift = F_x(np.array(grid)) * (w[-1]).T

                    # np.save("External drift contribution",external_drift)
                    # np.save("Gradient contribution",sigma_0_grad)
                    # np.save("Interface contribution",sigma_0_interf)

                    # print("External drift contribution:", np.max(external_drift), np.min(external_drift))
                    # print("Gradient contribution:", np.max(sigma_0_grad), np.min(sigma_0_grad))
                    # print("Interface contribution:", np.max(sigma_0_interf), np.min(sigma_0_interf))

                    interpolate_result = 1*sigma_0_grad+1*sigma_0_interf   + 1* external_drift
                    drift_result = 0*sigma_0_grad+0*sigma_0_interf + 1*external_drift

                print("weight:", w)
                intp = interpolate_result
                drift_contribution = drift_result
                if cv == True:
                    return intp
                else:
                    show_drift_var = self.show_drift_var.get()
                    if show_drift_var == 0:
                        intp = np.reshape(interpolate_result, [50, 50, 50])
                        mesh = plot_3D(grid, intp, surfaces_nr=2)
                        return intp, mesh
                    elif show_drift_var == 1:
                        drift_contribution = np.reshape(drift_contribution, [50, 50, 50])                    
                        mesh_drift = plot_3D_drift(grid, drift_contribution, surfaces_nr=10)                    
                        return drift_contribution, mesh_drift

            def systematic_sampling(df, n):
                count = len(df)
                step = max(1, count // n)
                indices = list(range(0, count, step))[:n]
                return df.iloc[indices]
            
            def kmeans_sampling_indices(df, n_samples,random_state=None):
                kmeans = KMeans(n_clusters=n_samples, random_state=random_state)
                kmeans.fit(df)
                
                sampled_indices = []
                for i in range(n_samples):
                    cluster_indices = (kmeans.labels_ == i)
                    cluster_points = df[cluster_indices]
                    
                    centroid = kmeans.cluster_centers_[i]
                    closest_index = cluster_points.apply(lambda row: ((row - centroid) ** 2).sum(), axis=1).idxmin()
                    sampled_indices.append(closest_index)
                
                return sampled_indices
            random_state = 24
            surface_sampled_indices = kmeans_sampling_indices(self.vertices_df, num_surface_points, random_state)

            if self.Sampling_method.get() == 'Statistic(K-means)':
                layer_position = self.vertices_df.iloc[surface_sampled_indices][['X', 'Y', 'Z']].values
                G_position = layer_position[:num_orientation_points]
                G_orientation = self.normals_o_df.iloc[surface_sampled_indices][['x', 'y', 'z']].values[:num_orientation_points]
            elif self.Sampling_method.get() == 'Random':
                layer_position = self.vertices_df.sample(n=num_surface_points, random_state=random_state)[['X', 'Y', 'Z']].values
                G_position = self.normals_df.sample(n=num_orientation_points, random_state=random_state)[['X', 'Y', 'Z']].values
                G_orientation = self.normals_o_df.sample(n=num_orientation_points, random_state=random_state)[['x', 'y', 'z']].values
            elif self.Sampling_method.get() == 'Systematic':
                layer_position = systematic_sampling(self.vertices_df, num_surface_points)[['X', 'Y', 'Z']].values
                G_position = systematic_sampling(self.normals_df, num_orientation_points)[['X', 'Y', 'Z']].values
                G_orientation = systematic_sampling(self.normals_o_df, num_orientation_points)[['x', 'y', 'z']].values

            start_time = time.time()
            intp, mesh = RBF_3D_kernel(G_position=G_position, 
                                                                        G_orientation=G_orientation,
                                                                        layer_position=layer_position, 
                                                                        test_data=None, drift=drift_type,
                                                                        cv=False) 
            cost_time = time.time() - start_time
            print('cost time:', cost_time)

            # Plot in a PyVista window
            def plot():
                show_drift_var = self.show_drift_var.get()
                only_show_data = self.only_show_data.get()
                plotter = pv.Plotter(notebook=False)
                if only_show_data == 0:
                    if show_drift_var == 0:
                        plotter.add_mesh(mesh,show_scalar_bar=False,color=model_color,opacity=0.8)
                    elif show_drift_var == 1:
                        plotter.add_mesh(mesh,show_scalar_bar=False,opacity=0.8)
                plotter.add_points(layer_position[:,[0,1,2]], render_points_as_spheres=True,point_size=14.0,color='blue')
                plotter.add_arrows((G_position[:,[0,1,2]]),direction=(G_orientation[:,[0,1,2]]),color='black',mag=50)
                plotter.set_background('white')
                plotter.show()
            plot_thread = threading.Thread(target=plot)
            plot_thread.start()

            # Prompt user to save the mesh
            file_path = filedialog.asksaveasfilename(
                initialfile=(
                    self.model_name + "_" +
                    self.Sampling_method.get() + "_" +
                    self.num_surface_points.get() + "_" +
                    self.num_orientation_points.get() + "_" +
                    self.kernel_function.get() + "_" +
                    self.drift_type.get() + "_" +
                    self.epsilon_entry.get() + "_" +
                    self.range_entry.get() + "_" +
                    self.param_a_entry.get() + "_" +
                    self.param_b_entry.get() + "_" +
                    self.param_c_entry.get() +
                    ".vtk"
                ),
                defaultextension=".vtk",
                filetypes=[("VTK files", "*.vtk")]
            )

            if file_path:
                file_path_scalar = file_path.replace(".vtk", "_scalar.npy")
                mesh.save(file_path)
                np.save(file_path_scalar, self.vtk_to_scalar(mesh))
                messagebox.showinfo("Save Mesh and Compute Scalar Field ", "Mesh and Scalar Field saved successfully!")
            

        except Exception as e:
            error_message = traceback.format_exc()
            messagebox.showerror("Error", f"An error occurred: {e}\n\n{error_message}")

    def load_vtk_to_gravity(self):
        file_path = filedialog.askopenfilename(filetypes=[("VTK files", "*.vtp"), ("VTK files", "*.vtk")])
        if file_path:
            self.mesh = pv.read(file_path)
            messagebox.showinfo("VTK Operations", "VTK file loaded successfully!")

    def my_point_gravity(self,coordinates, points, masses, field="g_z"):
        """
        Compute gravitational acceleration at given observation points due to point masses.

        Parameters:
        - coordinates: tuple of arrays (easting, northing, upward), each of shape (N,)
        - points: tuple of arrays (x_mass, y_mass, z_mass), each of shape (M,)
        - masses: array of shape (M,)
        - field: str, which component of the gravitational acceleration to compute ('g_z')

        Returns:
        - gravity: array of shape (N,), gravitational acceleration at observation points
        """
        G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2

        x_obs, y_obs, z_obs = coordinates
        x_mass, y_mass, z_mass = points

        # Ensure inputs are numpy arrays
        x_obs = np.asarray(x_obs)
        y_obs = np.asarray(y_obs)
        z_obs = np.asarray(z_obs)
        x_mass = np.asarray(x_mass)
        y_mass = np.asarray(y_mass)
        z_mass = np.asarray(z_mass)
        masses = np.asarray(masses)

        # Compute differences between observation points and masses
        dx = x_mass[np.newaxis, :] - x_obs[:, np.newaxis]  # Shape (N, M)
        dy = y_mass[np.newaxis, :] - y_obs[:, np.newaxis]
        dz = z_mass[np.newaxis, :] - z_obs[:, np.newaxis]
        r_squared = dx**2 + dy**2 + dz**2
        r = np.sqrt(r_squared)

        # Avoid division by zero
        r[r < 1e-10] = 1e-10

        # Compute gravitational acceleration components
        if field == 'g_z':
            # Vertical component
            g_contributions = G * masses / r**3 * dz
        elif field == 'g_x':
            # Easting component
            g_contributions = G * masses / r**3 * dx
        elif field == 'g_y':
            # Northing component
            g_contributions = G * masses / r**3 * dy
        else:
            raise ValueError(f"Unknown field component '{field}'")

        # Sum contributions from all masses
        gravity = np.sum(g_contributions, axis=1)

        return gravity
    
    def my_grid_coordinates(self,region, spacing):
        """
        Generate coordinate grids over a specified region with given spacing.

        Parameters:
        - region: tuple (xmin, xmax, ymin, ymax)
        - spacing: float, spacing between grid points

        Returns:
        - coordinates: tuple of arrays (easting, northing)
        """
        xmin, xmax, ymin, ymax = region
        # Create arrays of easting and northing coordinates
        easting = np.arange(xmin, xmax + spacing, spacing)
        northing = np.arange(ymin, ymax + spacing, spacing)
        # Create a meshgrid from the coordinate arrays
        easting, northing = np.meshgrid(easting, northing)
        return easting, northing

    def gravity_sim(self):
        geological_model = self.mesh
        points = geological_model.points
        xmin, xmax = np.min(points[:, 0]), np.max(points[:, 0])
        ymin, ymax = np.min(points[:, 1]), np.max(points[:, 1])
        density = 800  
        masses = np.full(points.shape[0], density)  
        coordinates = self.my_grid_coordinates(region=(xmin, xmax, ymin, ymax), spacing=100)
        easting, northing = coordinates
        upward = np.zeros_like(easting)  
        easting = easting.ravel()
        northing = northing.ravel()
        upward = upward.ravel()
        gravity = self.my_point_gravity(
            coordinates=(easting, northing, upward),
            points=(points[:, 0], points[:, 1], points[:, 2]),
            masses=masses,
            field="g_z"
        )
        return gravity, easting, northing
    
    def gravity_fwd_2d(self):
        gravity, easting, northing = self.gravity_sim()
        easting_mean = easting.mean()
        easting_std = easting.std()
        northing_mean = northing.mean()
        northing_std = northing.std()
        gravity_mean = gravity.mean()
        gravity_std = gravity.std()

        easting_norm = (easting - easting_mean) / easting_std
        northing_norm = (northing - northing_mean) / northing_std
        gravity_norm = (gravity - gravity_mean) / gravity_std
        def polynomial_gravity_anomaly(easting, northing, params):
            a, b, c, d, e, f = params
            gravity_anomaly = (
                a * easting**2 +
                b * northing**2 +
                c * easting * northing +
                d * easting +
                e * northing +
                f
            )
            return gravity_anomaly
        def misfit(params, easting, northing, gravity_observed, alpha=1e-6):
            gravity_calculated = polynomial_gravity_anomaly(easting, northing, params)
            residuals = gravity_observed - gravity_calculated
            regularization = alpha * np.sum(params**2)
            return np.sum(residuals**2) + regularization

        initial_params = np.zeros(6)

        result = minimize(
            misfit,
            initial_params,
            args=(easting_norm, northing_norm, gravity_norm),
            method='L-BFGS-B',
            options={'maxiter': 10000, 'ftol': 1e-12}
        )

        a_opt, b_opt, c_opt, d_opt, e_opt, f_opt = result.x

        gravity_calculated_norm = polynomial_gravity_anomaly(easting_norm, northing_norm, result.x)
        gravity_calculated = gravity_calculated_norm * gravity_std + gravity_mean
        self.result_label.config(
                    text = f'Best parameters:  a={a_opt:.2f}, b={b_opt:.2f}, c={c_opt:.2f}, d={d_opt:.2f}, e={e_opt:.2f}, f={f_opt:.2f}'
                )

        fig = plt.figure(figsize=(14,6))

        plt.subplot(1, 2, 1)
        plt.tricontourf(easting, northing, gravity, levels=50, cmap='viridis')
        plt.colorbar(label='Gravity disturbance (mGal)')
        plt.xlabel('Easting [m]')
        plt.ylabel('Northing [m]')
        plt.title('Observed Gravity Disturbance')

        plt.subplot(1, 2, 2)
        plt.tricontourf(easting, northing, gravity_calculated, levels=50, cmap='viridis')
        plt.colorbar(label='Gravity disturbance (mGal)')
        plt.xlabel('Easting [m]')
        plt.ylabel('Northing [m]')
        plt.title('Fitted Gravity Disturbance (2nd Order Polynomial)')
        plt.tight_layout()

        canvas4 = FigureCanvasTkAgg(fig, master=self.gravity_canvas_frame)
        canvas4.draw()
        canvas4.get_tk_widget().grid(row=4, column=0, padx=5, pady=5)

        return gravity_calculated, a_opt, b_opt, c_opt, d_opt, e_opt, f_opt

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Interpolation Method Comparison Tool") 
    app = GeologicalModelApp(root)
    app.run()
